# 🤖 Desarrollo de Modelos A/B Testing - Predicción Precio de Cierre t+1
## DeAcero Steel Price Predictor - Implementación Robusta de 15 Modelos

---

### 📋 **Objetivo Principal**
Predecir el **precio de cierre del día siguiente (t+1)** de la varilla corrugada LME mediante A/B testing robusto de 15 modelos diferentes.

### 🎯 **Estructura del Experimento**

**5 Arquitecturas de Modelo:**
1. **ARIMAX-GARCH**: Modelo econométrico con volatilidad condicional
2. **XGBoost**: Gradient Boosting para capturar no-linealidades
3. **LightGBM**: Gradient Boosting optimizado para velocidad
4. **Markov Regime-Switching VAR**: Modelo adaptativo de regímenes
5. **MIDAS**: Mixed Data Sampling para frecuencias mixtas

**3 Combinaciones de Variables:**
1. **Fundamental Pura** (5 variables): iron, coking, gas_natural, aluminio_lme, commodities
2. **Híbrida Balanceada** (6 variables): precio_varilla_lme_lag_1, volatility_20, iron, coking, commodities, VIX
3. **Régimen-Adaptativa** (6 variables): iron, coking, steel, VIX, sp500, tasa_interes_banxico

**Total: 5 × 3 = 15 modelos**

### 📊 **Metodología de Validación**
- **Time Series Cross-Validation**: 3 folds con walk-forward analysis
- **Búsqueda de Hiperparámetros**: GridSearchCV / Optuna
- **Métricas**: RMSE, MAE, MAPE, Directional Accuracy, Hit Rate (±2%)
- **Test Estadístico**: Diebold-Mariano para comparación significativa

---


In [1]:
# Configuración inicial y librerías
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Importar pandas y numpy primero
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import pickle
import joblib
from tqdm import tqdm

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Machine Learning
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

# Gradient Boosting
import xgboost as xgb
import lightgbm as lgb

# Time Series
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from arch import arch_model

# Optimización de hiperparámetros
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Configuración de estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("🚀 Librerías cargadas exitosamente")
print(f"📅 Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🐍 Python version: {sys.version.split()[0]}")


🚀 Librerías cargadas exitosamente
📅 Fecha de ejecución: 2025-09-29 23:37:12
🐍 Python version: 3.13.7


In [2]:
# Funciones auxiliares para métricas y evaluación

def calculate_directional_accuracy(y_true, y_pred):
    """Calcula la precisión direccional (si predice correctamente subida/bajada)"""
    y_true_diff = np.diff(y_true)
    y_pred_diff = np.diff(y_pred)
    return np.mean(np.sign(y_true_diff) == np.sign(y_pred_diff)) * 100

def calculate_hit_rate(y_true, y_pred, threshold=0.02):
    """Calcula el porcentaje de predicciones dentro del umbral especificado"""
    relative_error = np.abs((y_true - y_pred) / y_true)
    return np.mean(relative_error <= threshold) * 100

def calculate_sharpe_ratio(returns, risk_free_rate=0.02):
    """Calcula el Sharpe Ratio de una estrategia de trading"""
    excess_returns = returns - risk_free_rate/252  # Ajuste diario
    if np.std(excess_returns) == 0:
        return 0
    return np.sqrt(252) * np.mean(excess_returns) / np.std(excess_returns)

def diebold_mariano_test(e1, e2, h=1):
    """
    Diebold-Mariano test para comparar precisión de pronósticos
    H0: Los dos modelos tienen igual precisión
    """
    d = e1**2 - e2**2
    mean_d = np.mean(d)
    
    # Autocovarianza
    def autocovariance(xi, k):
        return np.mean((xi[:-k] - mean_d) * (xi[k:] - mean_d))
    
    # Varianza con corrección Newey-West
    gamma = [autocovariance(d, k) for k in range(h)]
    v_d = gamma[0] + 2 * sum(gamma[1:])
    
    # Estadístico DM
    dm_stat = mean_d / np.sqrt(v_d / len(d))
    
    # P-value (two-tailed)
    from scipy import stats
    p_value = 2 * (1 - stats.norm.cdf(abs(dm_stat)))
    
    return dm_stat, p_value

def evaluate_model(y_true, y_pred, model_name="Model"):
    """Evaluación completa del modelo con todas las métricas"""
    metrics = {
        'model': model_name,
        'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
        'mae': mean_absolute_error(y_true, y_pred),
        'mape': mean_absolute_percentage_error(y_true, y_pred) * 100,
        'r2': r2_score(y_true, y_pred),
        'directional_accuracy': calculate_directional_accuracy(y_true, y_pred),
        'hit_rate_2pct': calculate_hit_rate(y_true, y_pred, 0.02),
        'hit_rate_5pct': calculate_hit_rate(y_true, y_pred, 0.05)
    }
    return metrics

print("✅ Funciones de evaluación definidas")


✅ Funciones de evaluación definidas


In [3]:
# Carga de datos

# Rutas de datos
DATA_PATH = '../data/processed'
DAILY_DATA_PATH = f'{DATA_PATH}/daily_time_series/daily_series_consolidated_latest.csv'
MONTHLY_DATA_PATH = f'{DATA_PATH}/monthly_time_series/monthly_series_consolidated_latest.csv'

# Cargar datos diarios
print("📂 Cargando datos diarios...")
daily_data = pd.read_csv(DAILY_DATA_PATH)
for col in daily_data.columns:
    daily_data[col] = daily_data[col].interpolate(method='linear', limit_direction='both')
daily_data['fecha'] = pd.to_datetime(daily_data['fecha'])
daily_data.set_index('fecha', inplace=True)
daily_data = daily_data.sort_index()

print(f"✅ Datos diarios cargados: {daily_data.shape[0]} observaciones, {daily_data.shape[1]} variables")
print(f"   Período: {daily_data.index.min().date()} a {daily_data.index.max().date()}")

# Cargar datos mensuales
print("\n📂 Cargando datos mensuales...")
monthly_data = pd.read_csv(MONTHLY_DATA_PATH)

for col in monthly_data.columns:
    monthly_data[col] = monthly_data[col].interpolate(method='linear', limit_direction='both')
monthly_data['fecha'] = pd.to_datetime(monthly_data['fecha'])
monthly_data.set_index('fecha', inplace=True)
monthly_data = monthly_data.sort_index()

print(f"✅ Datos mensuales cargados: {monthly_data.shape[0]} observaciones, {monthly_data.shape[1]} variables")
print(f"   Período: {monthly_data.index.min().date()} a {monthly_data.index.max().date()}")

# Variable objetivo
TARGET_VAR = 'precio_varilla_lme'
print(f"\n🎯 Variable objetivo: {TARGET_VAR}")
print(f"   Rango de precios: ${daily_data[TARGET_VAR].min():.2f} - ${daily_data[TARGET_VAR].max():.2f}")
print(f"   Precio promedio: ${daily_data[TARGET_VAR].mean():.2f}")
print(f"   Volatilidad (std): ${daily_data[TARGET_VAR].std():.2f}")


📂 Cargando datos diarios...
✅ Datos diarios cargados: 1498 observaciones, 24 variables
   Período: 2020-01-02 a 2025-09-29

📂 Cargando datos mensuales...
✅ Datos mensuales cargados: 68 observaciones, 9 variables
   Período: 2020-01-01 a 2025-08-01

🎯 Variable objetivo: precio_varilla_lme
   Rango de precios: $408.50 - $590.70
   Precio promedio: $495.13
   Volatilidad (std): $30.57


In [4]:
# Feature Engineering y preparación de datos

def create_features(df, target_var=TARGET_VAR):
    """Crea features técnicas y de mercado"""
    features_df = df.copy()
    
    # 1. Lags de la variable objetivo
    for lag in [1, 2, 3, 5, 10, 20]:
        features_df[f'{target_var}_lag_{lag}'] = df[target_var].shift(lag)
    
    # 2. Medias móviles
    for window in [5, 10, 20, 50]:
        features_df[f'{target_var}_ma_{window}'] = df[target_var].rolling(window=window).mean()
    
    # 3. Volatilidad rolling
    for window in [5, 10, 20]:
        features_df[f'{target_var}_volatility_{window}'] = df[target_var].pct_change().rolling(window=window).std()
    
    # 4. Retornos
    features_df[f'{target_var}_return_1d'] = df[target_var].pct_change()
    features_df[f'{target_var}_return_5d'] = df[target_var].pct_change(5)
    features_df[f'{target_var}_return_20d'] = df[target_var].pct_change(20)
    
    # 5. RSI
    def calculate_rsi(prices, period=14):
        delta = prices.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        return rsi
    
    features_df[f'{target_var}_rsi_14'] = calculate_rsi(df[target_var])
    
    # 6. Bollinger Bands
    bb_window = 20
    bb_std = 2
    bb_ma = df[target_var].rolling(window=bb_window).mean()
    bb_std_val = df[target_var].rolling(window=bb_window).std()
    features_df[f'{target_var}_bb_upper'] = bb_ma + (bb_std_val * bb_std)
    features_df[f'{target_var}_bb_lower'] = bb_ma - (bb_std_val * bb_std)
    features_df[f'{target_var}_bb_width'] = features_df[f'{target_var}_bb_upper'] - features_df[f'{target_var}_bb_lower']
    features_df[f'{target_var}_bb_position'] = (df[target_var] - features_df[f'{target_var}_bb_lower']) / features_df[f'{target_var}_bb_width']
    
    # 7. Features de calendario
    features_df['day_of_week'] = df.index.dayofweek
    features_df['day_of_month'] = df.index.day
    features_df['month'] = df.index.month
    features_df['quarter'] = df.index.quarter
    
    return features_df

# Crear features
print("🔧 Creando features técnicas...")
daily_features = create_features(daily_data)
print(f"✅ Features creadas: {daily_features.shape[1]} variables totales")

# Eliminar filas con NaN (debido a lags y rolling windows)
daily_features = daily_features.dropna()
print(f"   Observaciones después de limpiar NaN: {daily_features.shape[0]}")


🔧 Creando features técnicas...
✅ Features creadas: 49 variables totales
   Observaciones después de limpiar NaN: 1449


In [5]:
# Definición de las 3 combinaciones de variables

# Combinación 1: FUNDAMENTAL PURA (5 variables)
fundamental_vars = [
    'iron',
    'coking', 
    'gas_natural',
    'aluminio_lme',
    'commodities'
]

# Combinación 2: HÍBRIDA BALANCEADA (6 variables + features técnicas)
hibrida_vars = [
    f'{TARGET_VAR}_lag_1',
    f'{TARGET_VAR}_volatility_20',
    'iron',
    'coking',
    'commodities',
    'VIX'
]

# Combinación 3: RÉGIMEN-ADAPTATIVA (6 variables)
regime_vars = [
    'iron',
    'coking',
    'steel',
    'VIX',
    'sp500',
    'tasa_interes_banxico'
]

# Diccionario de combinaciones
variable_combinations = {
    'fundamental': fundamental_vars,
    'hibrida': hibrida_vars,
    'regime': regime_vars
}

print("📊 Combinaciones de variables definidas:")
for name, vars in variable_combinations.items():
    print(f"\n   {name.upper()}:")
    for var in vars:
        if var in daily_features.columns:
            print(f"      ✅ {var}")
        else:
            print(f"      ❌ {var} (no encontrada)")


📊 Combinaciones de variables definidas:

   FUNDAMENTAL:
      ✅ iron
      ✅ coking
      ✅ gas_natural
      ✅ aluminio_lme
      ✅ commodities

   HIBRIDA:
      ✅ precio_varilla_lme_lag_1
      ✅ precio_varilla_lme_volatility_20
      ✅ iron
      ✅ coking
      ✅ commodities
      ✅ VIX

   REGIME:
      ✅ iron
      ✅ coking
      ✅ steel
      ✅ VIX
      ✅ sp500
      ✅ tasa_interes_banxico


In [6]:
# Preparación de datos para modelado

def prepare_data_for_modeling(df, target_var, feature_vars, forecast_horizon=1):
    """
    Prepara los datos para modelado con predicción t+forecast_horizon
    """
    # Seleccionar features
    X = df[feature_vars].copy()
    
    # Variable objetivo: precio en t+forecast_horizon
    y = df[target_var].shift(-forecast_horizon)
    
    # Eliminar NaN
    valid_idx = ~(X.isna().any(axis=1) | y.isna())
    X = X[valid_idx]
    y = y[valid_idx]
    
    # Convertir a retornos logarítmicos para estacionariedad
    X_returns = pd.DataFrame(index=X.index)
    for col in X.columns:
        if col.endswith('_lag_1') or 'volatility' in col or 'VIX' in col or 'tasa_interes' in col:
            # Mantener en niveles
            X_returns[col] = X[col]
        else:
            # Convertir a retornos logarítmicos
            X_returns[col] = np.log(X[col] / X[col].shift(1))
    
    # Target: retorno logarítmico
    y_returns = np.log(y / df[target_var][valid_idx])
    
    # Eliminar infinitos y NaN resultantes
    valid_returns = ~(X_returns.isna().any(axis=1) | y_returns.isna() | 
                     np.isinf(X_returns).any(axis=1) | np.isinf(y_returns))
    
    X_clean = X_returns[valid_returns]
    y_clean = y_returns[valid_returns]
    
    return X_clean, y_clean

# Preparar datos para cada combinación
print("📦 Preparando datos para cada combinación de variables...\n")

prepared_data = {}
for combo_name, vars_list in variable_combinations.items():
    print(f"Procesando {combo_name}...")
    try:
        X, y = prepare_data_for_modeling(daily_features, TARGET_VAR, vars_list)
        prepared_data[combo_name] = {'X': X, 'y': y}
        print(f"   ✅ Shape: X={X.shape}, y={y.shape}")
        print(f"   Período: {X.index.min().date()} a {X.index.max().date()}")
    except Exception as e:
        print(f"   ❌ Error: {e}")
        
print("\n✅ Datos preparados para modelado")


📦 Preparando datos para cada combinación de variables...

Procesando fundamental...
   ✅ Shape: X=(1447, 5), y=(1447,)
   Período: 2020-03-12 a 2025-09-26
Procesando hibrida...
   ✅ Shape: X=(1447, 6), y=(1447,)
   Período: 2020-03-12 a 2025-09-26
Procesando regime...
   ✅ Shape: X=(1447, 6), y=(1447,)
   Período: 2020-03-12 a 2025-09-26

✅ Datos preparados para modelado


In [7]:
# Configuración de Time Series Cross-Validation

class TimeSeriesCV:
    """Validación cruzada específica para series temporales con walk-forward analysis"""
    
    def __init__(self, n_splits=3, train_size=500, test_size=60, gap=0):
        self.n_splits = n_splits
        self.train_size = train_size
        self.test_size = test_size
        self.gap = gap
    
    def split(self, X, y=None):
        n_samples = len(X)
        indices = np.arange(n_samples)
        
        # Calcular el paso entre splits
        step = (n_samples - self.train_size - self.test_size - self.gap) // (self.n_splits - 1)
        
        for i in range(self.n_splits):
            train_start = i * step
            train_end = train_start + self.train_size
            test_start = train_end + self.gap
            test_end = min(test_start + self.test_size, n_samples)
            
            if test_end > n_samples:
                break
                
            train_idx = indices[train_start:train_end]
            test_idx = indices[test_start:test_end]
            
            yield train_idx, test_idx

# Crear objeto de validación cruzada
tscv = TimeSeriesCV(n_splits=3, train_size=500, test_size=60, gap=0)

# Visualizar los splits
print("📊 Configuración de Time Series Cross-Validation:")
print(f"   - Número de splits: 3")
print(f"   - Tamaño de entrenamiento: 500 días")
print(f"   - Tamaño de prueba: 60 días")
print(f"   - Gap entre train y test: 0 días\n")

# Mostrar ejemplo de splits para la primera combinación
if prepared_data:
    first_combo = list(prepared_data.keys())[0]
    X_example = prepared_data[first_combo]['X']
    y_example = prepared_data[first_combo]['y']
    
    print(f"Ejemplo de splits para combinación '{first_combo}':")
    for i, (train_idx, test_idx) in enumerate(tscv.split(X_example)):
        train_dates = X_example.iloc[train_idx].index
        test_dates = X_example.iloc[test_idx].index
        print(f"\n   Split {i+1}:")
        print(f"      Train: {train_dates.min().date()} a {train_dates.max().date()} ({len(train_idx)} días)")
        print(f"      Test:  {test_dates.min().date()} a {test_dates.max().date()} ({len(test_idx)} días)")


📊 Configuración de Time Series Cross-Validation:
   - Número de splits: 3
   - Tamaño de entrenamiento: 500 días
   - Tamaño de prueba: 60 días
   - Gap entre train y test: 0 días

Ejemplo de splits para combinación 'fundamental':

   Split 1:
      Train: 2020-03-12 a 2022-02-09 (500 días)
      Test:  2022-02-10 a 2022-05-04 (60 días)

   Split 2:
      Train: 2021-11-23 a 2023-10-23 (500 días)
      Test:  2023-10-24 a 2024-01-15 (60 días)

   Split 3:
      Train: 2023-08-04 a 2025-07-03 (500 días)
      Test:  2025-07-04 a 2025-09-25 (60 días)


## 🤖 Implementación de los 5 Modelos

### Modelo 1: ARIMAX-GARCH

In [8]:
# Modelo 1: ARIMAX-GARCH

def train_arimax_garch_model(X_train, y_train, X_test, y_test, combo_name):
    """
    Entrena modelo ARIMAX con componente GARCH para volatilidad
    """
    from statsmodels.tsa.statespace.sarimax import SARIMAX
    
    # Preparar datos para ARIMAX
    # Combinar X e y para mantener alineación temporal
    train_data = pd.concat([y_train, X_train], axis=1)
    test_data = pd.concat([y_test, X_test], axis=1)
    
    # Buscar mejores órdenes ARIMA con AIC
    best_aic = np.inf
    best_order = None
    best_model = None
    
    # Grid search simplificado para órdenes ARIMA
    p_values = [0, 1, 2]
    d_values = [0, 1]
    q_values = [0, 1, 2]
    
    for p in p_values:
        for d in d_values:
            for q in q_values:
                try:
                    model = SARIMAX(y_train, 
                                   exog=X_train,
                                   order=(p, d, q),
                                   enforce_stationarity=False,
                                   enforce_invertibility=False)
                    
                    fitted = model.fit(disp=False)
                    
                    if fitted.aic < best_aic:
                        best_aic = fitted.aic
                        best_order = (p, d, q)
                        best_model = fitted
                        
                except Exception:
                    continue
    
    # Si encontramos un modelo válido
    if best_model is not None:
        # Predicciones ARIMAX
        y_pred_train = best_model.fittedvalues
        y_pred_test = best_model.forecast(steps=len(X_test), exog=X_test)
        
        # Residuos para GARCH
        residuals = y_train - y_pred_train
        
        # Ajustar modelo GARCH(1,1) a los residuos
        try:
            garch_model = arch_model(residuals, vol='Garch', p=1, q=1, dist='t')
            garch_fitted = garch_model.fit(disp='off')
            
            # Pronóstico de volatilidad
            volatility_forecast = garch_fitted.forecast(horizon=len(X_test))
            
            # Intervalos de confianza usando volatilidad GARCH
            vol_test = np.sqrt(volatility_forecast.variance.values[-1, :])
            
        except:
            # Si GARCH falla, usar volatilidad constante
            vol_test = np.std(residuals) * np.ones(len(X_test))
    else:
        # Modelo de respaldo si ARIMAX falla
        y_pred_train = np.mean(y_train) * np.ones(len(y_train))
        y_pred_test = np.mean(y_train) * np.ones(len(y_test))
        best_order = (0, 0, 0)
        vol_test = np.std(y_train) * np.ones(len(y_test))
    
    # Métricas
    train_metrics = evaluate_model(y_train.values, y_pred_train, f"ARIMAX-GARCH_{combo_name}_train")
    test_metrics = evaluate_model(y_test.values, y_pred_test, f"ARIMAX-GARCH_{combo_name}_test")
    
    return {
        'model': best_model,
        'arima_order': best_order,
        'train_metrics': train_metrics,
        'test_metrics': test_metrics,
        'predictions': {'train': y_pred_train, 'test': y_pred_test},
        'volatility_forecast': vol_test if 'vol_test' in locals() else None
    }

print("✅ Función ARIMAX-GARCH definida")


✅ Función ARIMAX-GARCH definida


### Modelo 2: XGBoost

In [9]:
# Modelo 2: XGBoost con Optuna para búsqueda de hiperparámetros

def train_xgboost_model(X_train, y_train, X_test, y_test, combo_name, n_trials=50):
    """
    Entrena modelo XGBoost con búsqueda de hiperparámetros usando Optuna
    """
    
    def objective(trial):
        # Hiperparámetros a optimizar
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'gamma': trial.suggest_float('gamma', 0, 0.5),
            'reg_alpha': trial.suggest_float('reg_alpha', 0, 1.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0, 1.0),
            'random_state': 42,
            'n_jobs': -1
        }
        
        # Crear modelo sin early_stopping_rounds para la optimización
        model = xgb.XGBRegressor(**params)
        
        # Validación cruzada con TimeSeriesCV
        cv_scores = []
        for train_idx, val_idx in tscv.split(X_train):
            X_cv_train = X_train.iloc[train_idx]
            y_cv_train = y_train.iloc[train_idx]
            X_cv_val = X_train.iloc[val_idx]
            y_cv_val = y_train.iloc[val_idx]
            
            # Entrenar sin early stopping en la validación cruzada
            model.fit(X_cv_train, y_cv_train, verbose=False)
            
            y_pred = model.predict(X_cv_val)
            rmse = np.sqrt(mean_squared_error(y_cv_val, y_pred))
            cv_scores.append(rmse)
        
        return np.mean(cv_scores)
    
    # Crear estudio de Optuna
    study = optuna.create_study(direction='minimize', study_name=f'xgboost_{combo_name}')
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
    
    # Entrenar modelo final con mejores hiperparámetros
    best_params = study.best_params
    best_params['random_state'] = 42
    best_params['n_jobs'] = -1
    
    # Para el modelo final, usar early stopping con un conjunto de validación
    X_train_split = X_train.iloc[:-int(len(X_train)*0.2)]
    y_train_split = y_train.iloc[:-int(len(y_train)*0.2)]
    X_val_split = X_train.iloc[-int(len(X_train)*0.2):]
    y_val_split = y_train.iloc[-int(len(y_train)*0.2):]
    
    # Agregar early stopping solo para el modelo final
    best_params['early_stopping_rounds'] = 50
    
    final_model = xgb.XGBRegressor(**best_params)
    final_model.fit(X_train_split, y_train_split, 
                   eval_set=[(X_val_split, y_val_split)],
                   verbose=False)
    
    # Predicciones
    y_pred_train = final_model.predict(X_train)
    y_pred_test = final_model.predict(X_test)
    
    # Métricas
    train_metrics = evaluate_model(y_train, y_pred_train, f"XGBoost_{combo_name}_train")
    test_metrics = evaluate_model(y_test, y_pred_test, f"XGBoost_{combo_name}_test")
    
    return {
        'model': final_model,
        'best_params': best_params,
        'train_metrics': train_metrics,
        'test_metrics': test_metrics,
        'predictions': {'train': y_pred_train, 'test': y_pred_test}
    }

print("✅ Función XGBoost definida")


✅ Función XGBoost definida


### Modelo 3: LightGBM

In [10]:
# Modelo 3: LightGBM con Optuna

def train_lightgbm_model(X_train, y_train, X_test, y_test, combo_name, n_trials=50):
    """
    Entrena modelo LightGBM con búsqueda de hiperparámetros usando Optuna
    """
    
    def objective(trial):
        # Hiperparámetros a optimizar
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'num_leaves': trial.suggest_int('num_leaves', 10, 100),
            'max_depth': trial.suggest_int('max_depth', 3, 15),
            'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True),
            'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
            'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
            'reg_alpha': trial.suggest_float('reg_alpha', 0, 1.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0, 1.0),
            'random_state': 42,
            'n_jobs': -1,
            'verbosity': -1
        }
        
        # Crear y entrenar modelo
        model = lgb.LGBMRegressor(**params)
        
        # Validación cruzada con TimeSeriesCV
        cv_scores = []
        for train_idx, val_idx in tscv.split(X_train):
            X_cv_train = X_train.iloc[train_idx]
            y_cv_train = y_train.iloc[train_idx]
            X_cv_val = X_train.iloc[val_idx]
            y_cv_val = y_train.iloc[val_idx]
            
            model.fit(X_cv_train, y_cv_train,
                     eval_set=[(X_cv_val, y_cv_val)],
                     callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)])
            
            y_pred = model.predict(X_cv_val)
            rmse = np.sqrt(mean_squared_error(y_cv_val, y_pred))
            cv_scores.append(rmse)
        
        return np.mean(cv_scores)
    
    # Crear estudio de Optuna
    study = optuna.create_study(direction='minimize', study_name=f'lightgbm_{combo_name}')
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
    
    # Entrenar modelo final con mejores hiperparámetros
    best_params = study.best_params
    best_params['random_state'] = 42
    best_params['n_jobs'] = -1
    best_params['verbosity'] = -1
    
    final_model = lgb.LGBMRegressor(**best_params)
    final_model.fit(X_train, y_train)
    
    # Predicciones
    y_pred_train = final_model.predict(X_train)
    y_pred_test = final_model.predict(X_test)
    
    # Métricas
    train_metrics = evaluate_model(y_train, y_pred_train, f"LightGBM_{combo_name}_train")
    test_metrics = evaluate_model(y_test, y_pred_test, f"LightGBM_{combo_name}_test")
    
    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': final_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    return {
        'model': final_model,
        'best_params': best_params,
        'train_metrics': train_metrics,
        'test_metrics': test_metrics,
        'predictions': {'train': y_pred_train, 'test': y_pred_test},
        'feature_importance': feature_importance
    }

print("✅ Función LightGBM definida")


✅ Función LightGBM definida


### Modelo 4: Markov Regime-Switching VAR

In [11]:
# Modelo 4: Markov Regime-Switching VAR (simplificado)

def train_regime_switching_model(X_train, y_train, X_test, y_test, combo_name):
    """
    Entrena modelo de cambio de régimen de Markov
    Versión simplificada usando clustering para detectar regímenes
    """
    from sklearn.cluster import KMeans
    from sklearn.mixture import GaussianMixture
    
    # Detectar regímenes usando GMM en los retornos y volatilidad
    returns = y_train.pct_change().dropna()
    volatility = returns.rolling(window=20).std().dropna()
    
    # Crear features para detección de régimen
    regime_features = pd.DataFrame({
        'returns': returns[volatility.index],
        'volatility': volatility
    }).dropna()
    
    # Ajustar GMM con 3 regímenes
    n_regimes = 3
    gmm = GaussianMixture(n_components=n_regimes, random_state=42)
    
    if len(regime_features) > 0:
        regimes_train = gmm.fit_predict(regime_features)
        
        # Expandir regímenes a todo el conjunto de entrenamiento
        regime_series = pd.Series(index=y_train.index)
        regime_series.iloc[len(y_train) - len(regimes_train):] = regimes_train
        regime_series = regime_series.fillna(method='bfill').fillna(0)
        
        # Entrenar un modelo separado para cada régimen
        models_by_regime = {}
        
        for regime in range(n_regimes):
            mask = regime_series == regime
            if mask.sum() > 10:  # Mínimo de observaciones
                X_regime = X_train[mask]
                y_regime = y_train[mask]
                
                # Usar XGBoost para cada régimen
                model = xgb.XGBRegressor(
                    n_estimators=100,
                    max_depth=5,
                    learning_rate=0.1,
                    random_state=42
                )
                model.fit(X_regime, y_regime)
                models_by_regime[regime] = model
        
        # Para predicción, detectar régimen actual y usar modelo correspondiente
        if len(models_by_regime) > 0:
            # Detectar régimen para test
            test_returns = y_test.pct_change().fillna(0)
            test_volatility = test_returns.rolling(window=min(20, len(test_returns))).std().fillna(0)
            
            test_regime_features = pd.DataFrame({
                'returns': test_returns,
                'volatility': test_volatility
            })
            
            regimes_test = gmm.predict(test_regime_features)
            
            # Predicciones por régimen
            y_pred_train = np.zeros(len(y_train))
            y_pred_test = np.zeros(len(y_test))
            
            # Train predictions
            for regime in models_by_regime:
                mask = regime_series == regime
                if mask.sum() > 0:
                    y_pred_train[mask] = models_by_regime[regime].predict(X_train[mask])
            
            # Test predictions
            for i, regime in enumerate(regimes_test):
                if regime in models_by_regime:
                    y_pred_test[i] = models_by_regime[regime].predict(X_test.iloc[[i]])
                else:
                    # Usar modelo del régimen más común si no existe
                    default_regime = regime_series.mode()[0]
                    if default_regime in models_by_regime:
                        y_pred_test[i] = models_by_regime[default_regime].predict(X_test.iloc[[i]])
        else:
            # Fallback a predicción simple
            y_pred_train = np.mean(y_train) * np.ones(len(y_train))
            y_pred_test = np.mean(y_train) * np.ones(len(y_test))
    else:
        # Fallback si no hay suficientes datos
        y_pred_train = np.mean(y_train) * np.ones(len(y_train))
        y_pred_test = np.mean(y_train) * np.ones(len(y_test))
        regimes_train = np.zeros(len(y_train))
        regimes_test = np.zeros(len(y_test))
    
    # Métricas
    train_metrics = evaluate_model(y_train.values, y_pred_train, f"RegimeSwitching_{combo_name}_train")
    test_metrics = evaluate_model(y_test.values, y_pred_test, f"RegimeSwitching_{combo_name}_test")
    
    return {
        'model': {'gmm': gmm, 'models_by_regime': models_by_regime if 'models_by_regime' in locals() else {}},
        'train_metrics': train_metrics,
        'test_metrics': test_metrics,
        'predictions': {'train': y_pred_train, 'test': y_pred_test},
        'regimes': {'train': regimes_train if 'regimes_train' in locals() else None, 
                   'test': regimes_test if 'regimes_test' in locals() else None}
    }

print("✅ Función Markov Regime-Switching definida")


✅ Función Markov Regime-Switching definida


### Modelo 5: MIDAS (Mixed Data Sampling)

In [12]:
# Modelo 5: MIDAS (Mixed Data Sampling) - Implementación simplificada

def train_midas_model(X_train, y_train, X_test, y_test, combo_name, monthly_data):
    """
    Entrena modelo MIDAS para combinar frecuencias diarias y mensuales
    Implementación simplificada usando weighted regression
    """
    
    # Alinear datos mensuales con datos diarios
    # Resamplear datos mensuales a diarios con forward fill
    monthly_resampled = monthly_data.resample('D').ffill()
    
    # Alinear con índice de X_train
    monthly_aligned_train = monthly_resampled.reindex(X_train.index, method='ffill')
    monthly_aligned_test = monthly_resampled.reindex(X_test.index, method='ffill')
    
    # Seleccionar columnas mensuales relevantes
    monthly_cols = ['inflacion_mensual_mexico', 'produccion_industrial_usa', 
                   'produccion_metalurgica_mexico']
    
    # Filtrar columnas que existen
    monthly_cols = [col for col in monthly_cols if col in monthly_aligned_train.columns]
    
    if len(monthly_cols) > 0:
        # Crear ponderaciones exponenciales para datos mensuales (Almon polynomial)
        def almon_weights(n_lags, theta1=0.1, theta2=-0.01):
            """Genera ponderaciones Almon para MIDAS"""
            lags = np.arange(n_lags)
            weights = np.exp(theta1 * lags + theta2 * lags**2)
            return weights / weights.sum()
        
        # Crear features ponderadas de datos mensuales
        n_lags = 30  # 30 días de historia
        weights = almon_weights(n_lags)
        
        # Aplicar ponderaciones a variables mensuales
        midas_features_train = pd.DataFrame(index=X_train.index)
        midas_features_test = pd.DataFrame(index=X_test.index)
        
        for col in monthly_cols:
            if col in monthly_aligned_train.columns:
                # Crear lags ponderados
                weighted_col = np.zeros(len(X_train))
                for i in range(len(X_train)):
                    if i >= n_lags:
                        values = monthly_aligned_train[col].iloc[i-n_lags:i].values
                        if len(values) == n_lags and not np.isnan(values).any():
                            weighted_col[i] = np.sum(values * weights)
                        else:
                            weighted_col[i] = monthly_aligned_train[col].iloc[i] if i < len(monthly_aligned_train) else 0
                    else:
                        weighted_col[i] = monthly_aligned_train[col].iloc[i] if i < len(monthly_aligned_train) else 0
                
                midas_features_train[f'midas_{col}'] = weighted_col
                
                # Mismo proceso para test
                weighted_col_test = np.zeros(len(X_test))
                for i in range(len(X_test)):
                    values = monthly_aligned_test[col].iloc[max(0, i-n_lags):i].values
                    if len(values) > 0 and not np.isnan(values).all():
                        if len(values) < n_lags:
                            # Padding con el último valor si no hay suficientes lags
                            padded_values = np.pad(values, (n_lags - len(values), 0), 'edge')
                            weighted_col_test[i] = np.sum(padded_values * weights)
                        else:
                            weighted_col_test[i] = np.sum(values[-n_lags:] * weights)
                    else:
                        weighted_col_test[i] = monthly_aligned_test[col].iloc[i] if i < len(monthly_aligned_test) else 0
                
                midas_features_test[f'midas_{col}'] = weighted_col_test
        
        # Combinar features diarias con MIDAS features
        X_train_midas = pd.concat([X_train, midas_features_train], axis=1)
        X_test_midas = pd.concat([X_test, midas_features_test], axis=1)
        
        # Limpiar NaN e infinitos
        X_train_midas = X_train_midas.replace([np.inf, -np.inf], np.nan).fillna(0)
        X_test_midas = X_test_midas.replace([np.inf, -np.inf], np.nan).fillna(0)
        
    else:
        # Si no hay datos mensuales, usar solo datos diarios
        X_train_midas = X_train
        X_test_midas = X_test
    
    # Entrenar modelo XGBoost con features MIDAS
    model = xgb.XGBRegressor(
        n_estimators=300,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    
    model.fit(X_train_midas, y_train)
    
    # Predicciones
    y_pred_train = model.predict(X_train_midas)
    y_pred_test = model.predict(X_test_midas)
    
    # Métricas
    train_metrics = evaluate_model(y_train.values, y_pred_train, f"MIDAS_{combo_name}_train")
    test_metrics = evaluate_model(y_test.values, y_pred_test, f"MIDAS_{combo_name}_test")
    
    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': X_train_midas.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    return {
        'model': model,
        'train_metrics': train_metrics,
        'test_metrics': test_metrics,
        'predictions': {'train': y_pred_train, 'test': y_pred_test},
        'feature_importance': feature_importance,
        'midas_features': list(midas_features_train.columns) if 'midas_features_train' in locals() else []
    }

print("✅ Función MIDAS definida")


✅ Función MIDAS definida


## 🚀 Ejecución del A/B Testing - 15 Modelos


In [13]:
# Ejecutar todos los modelos con todas las combinaciones de variables

# Dividir datos en train/test (80/20 del período total)
def split_time_series_data(X, y, test_size=0.05):
    """Divide datos de series temporales manteniendo orden temporal"""
    n_samples = len(X)
    split_idx = int(n_samples * (1 - test_size))
    
    X_train = X.iloc[:split_idx]
    X_test = X.iloc[split_idx:]
    y_train = y.iloc[:split_idx]
    y_test = y.iloc[split_idx:]
    
    return X_train, X_test, y_train, y_test

# Función para limpiar datos de valores infinitos y NaN
def clean_data_for_training(X, y):
    """Limpia datos de valores infinitos, NaN y valores extremos"""
    # Crear copias para no modificar originales
    X_clean = X.copy()
    y_clean = y.copy()
    
    # Reemplazar infinitos con NaN
    X_clean = X_clean.replace([np.inf, -np.inf], np.nan)
    
    # Identificar filas con NaN en X o y
    mask_x = ~X_clean.isnull().any(axis=1)
    mask_y = ~y_clean.isnull()
    mask_combined = mask_x & mask_y
    
    # Filtrar datos
    X_clean = X_clean[mask_combined]
    y_clean = y_clean[mask_combined]
    
    # Detectar y manejar valores extremos (outliers)
    for col in X_clean.select_dtypes(include=[np.number]).columns:
        Q1 = X_clean[col].quantile(0.01)
        Q3 = X_clean[col].quantile(0.99)
        IQR = Q3 - Q1
        lower_bound = Q1 - 3 * IQR
        upper_bound = Q3 + 3 * IQR
        
        # Clip valores extremos
        X_clean[col] = X_clean[col].clip(lower_bound, upper_bound)
    
    # Verificar que no hay valores infinitos o NaN restantes
    assert not X_clean.isnull().any().any(), "Aún hay valores NaN en X"
    assert not y_clean.isnull().any(), "Aún hay valores NaN en y"
    assert not np.isinf(X_clean.values).any(), "Aún hay valores infinitos en X"
    assert not np.isinf(y_clean.values).any(), "Aún hay valores infinitos en y"
    
    print(f"   🧹 Datos limpiados: {len(X_clean)} observaciones válidas de {len(X)} originales")
    
    return X_clean, y_clean

# Almacenar todos los resultados
all_results = []

# Lista de modelos a entrenar
models_to_train = [
    ('ARIMAX-GARCH', train_arimax_garch_model),
    ('XGBoost', train_xgboost_model),
    ('LightGBM', train_lightgbm_model),
    ('RegimeSwitching', train_regime_switching_model),
    ('MIDAS', train_midas_model)
]

print("🚀 INICIANDO A/B TESTING DE 15 MODELOS\n")
print("=" * 60)

# Iterar sobre cada combinación de variables
for combo_name, combo_data in prepared_data.items():
    print(f"\n📊 COMBINACIÓN: {combo_name.upper()}")
    print("-" * 40)
    
    X = combo_data['X']
    y = combo_data['y']
    
    # Limpiar datos antes del split
    try:
        X_clean, y_clean = clean_data_for_training(X, y)
    except Exception as e:
        print(f"   ❌ Error limpiando datos: {str(e)}")
        continue
    
    # Split train/test con datos limpios
    X_train, X_test, y_train, y_test = split_time_series_data(X_clean, y_clean)
    
    print(f"   Train: {X_train.shape[0]} observaciones")
    print(f"   Test:  {X_test.shape[0]} observaciones\n")
    
    # Entrenar cada modelo
    for model_name, train_function in models_to_train:
        print(f"   🤖 Entrenando {model_name}...")
        
        try:
            # Verificación adicional antes de entrenar
            if np.isinf(X_train.values).any() or np.isinf(X_test.values).any():
                raise ValueError(f"Valores infinitos detectados en datos de entrada para {model_name}")
            
            if np.isinf(y_train.values).any() or np.isinf(y_test.values).any():
                raise ValueError(f"Valores infinitos detectados en variable objetivo para {model_name}")
            
            # Ajustar parámetros según el modelo
            if model_name == 'XGBoost':
                result = train_function(X_train, y_train, X_test, y_test, combo_name, n_trials=20)
            elif model_name == 'LightGBM':
                result = train_function(X_train, y_train, X_test, y_test, combo_name, n_trials=20)
            elif model_name == 'MIDAS':
                result = train_function(X_train, y_train, X_test, y_test, combo_name, monthly_data)
            else:
                result = train_function(X_train, y_train, X_test, y_test, combo_name)
            
            # Guardar resultados
            all_results.append({
                'model': model_name,
                'combination': combo_name,
                'train_rmse': result['train_metrics']['rmse'],
                'test_rmse': result['test_metrics']['rmse'],
                'train_mae': result['train_metrics']['mae'],
                'test_mae': result['test_metrics']['mae'],
                'train_mape': result['train_metrics']['mape'],
                'test_mape': result['test_metrics']['mape'],
                'train_r2': result['train_metrics']['r2'],
                'test_r2': result['test_metrics']['r2'],
                'directional_accuracy': result['test_metrics']['directional_accuracy'],
                'hit_rate_2pct': result['test_metrics']['hit_rate_2pct'],
                'hit_rate_5pct': result['test_metrics']['hit_rate_5pct'],
                'full_result': result
            })
            
            print(f"      ✅ RMSE Test: {result['test_metrics']['rmse']:.4f}")
            print(f"      ✅ MAPE Test: {result['test_metrics']['mape']:.2f}%")
            print(f"      ✅ Dir. Acc: {result['test_metrics']['directional_accuracy']:.1f}%")
            print(f"      ✅ Hit Rate ±2%: {result['test_metrics']['hit_rate_2pct']:.1f}%\n")
            
        except Exception as e:
            print(f"      ❌ Error: {str(e)[:100]}...\n")
            
            # Guardar resultado con error
            all_results.append({
                'model': model_name,
                'combination': combo_name,
                'error': str(e),
                'train_rmse': np.nan,
                'test_rmse': np.nan,
                'train_mae': np.nan,
                'test_mae': np.nan,
                'train_mape': np.nan,
                'test_mape': np.nan,
                'train_r2': np.nan,
                'test_r2': np.nan,
                'directional_accuracy': np.nan,
                'hit_rate_2pct': np.nan,
                'hit_rate_5pct': np.nan
            })

print("\n" + "=" * 60)
print("✅ A/B TESTING COMPLETADO")
print(f"   Total de modelos entrenados: {len(all_results)}")
print("=" * 60)


🚀 INICIANDO A/B TESTING DE 15 MODELOS


📊 COMBINACIÓN: FUNDAMENTAL
----------------------------------------
   🧹 Datos limpiados: 1447 observaciones válidas de 1447 originales
   Train: 1374 observaciones
   Test:  73 observaciones

   🤖 Entrenando ARIMAX-GARCH...


/Users/franciscojavierriverapaleo/test_gerencia/deacero_env/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency B will be used.
  self._init_dates(dates, freq)
/Users/franciscojavierriverapaleo/test_gerencia/deacero_env/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency B will be used.
  self._init_dates(dates, freq)
/Users/franciscojavierriverapaleo/test_gerencia/deacero_env/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency B will be used.
  self._init_dates(dates, freq)
/Users/franciscojavierriverapaleo/test_gerencia/deacero_env/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency B will be used.
  self._init_dates(dates, fr

      ✅ RMSE Test: 0.0373
      ✅ MAPE Test: 480525111470.10%
      ✅ Dir. Acc: 40.3%
      ✅ Hit Rate ±2%: 0.0%

   🤖 Entrenando XGBoost...


/Users/franciscojavierriverapaleo/test_gerencia/deacero_env/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


  0%|          | 0/20 [00:00<?, ?it/s]

...

/Users/franciscojavierriverapaleo/test_gerencia/deacero_env/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


  0%|          | 0/20 [00:00<?, ?it/s]

      ✅ RMSE Test: 0.0373
      ✅ MAPE Test: 2319075030716.49%
      ✅ Dir. Acc: 1.4%
      ✅ Hit Rate ±2%: 0.0%

   🤖 Entrenando LightGBM...


  0%|          | 0/20 [00:00<?, ?it/s]

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[175]	valid_0's l2: 0.001245
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[5]	valid_0's l2: 0.00162065
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[2]	valid_0's l2: 0.00121912
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[36]	valid_0's l2: 0.00120772
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[29]	valid_0's l2: 0.00160738
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[2]	valid_0's l2: 0.00121921
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[27]	valid_0's l2: 0.0012366
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.0016217

In [14]:
# DIAGNÓSTICO: Analizar las predicciones

print("🔍 DIAGNÓSTICO DEL PROBLEMA DE PREDICCIÓN")
print("=" * 60)

# Verificar la distribución de la variable objetivo
if prepared_data:
    for combo_name, combo_data in prepared_data.items():
        y = combo_data['y']
        print(f"\n📊 Combinación: {combo_name}")
        print(f"   Estadísticas de y (retornos logarítmicos):")
        print(f"   - Media: {y.mean():.6f}")
        print(f"   - Std: {y.std():.6f}")
        print(f"   - Min: {y.min():.6f}")
        print(f"   - Max: {y.max():.6f}")
        print(f"   - Mediana: {y.median():.6f}")
        
        # Verificar si hay valores extremos
        q1 = y.quantile(0.25)
        q3 = y.quantile(0.75)
        iqr = q3 - q1
        outliers = ((y < (q1 - 1.5 * iqr)) | (y > (q3 + 1.5 * iqr))).sum()
        print(f"   - Outliers: {outliers} ({outliers/len(y)*100:.1f}%)")
        
        # Verificar autocorrelación
        from statsmodels.stats.diagnostic import acorr_ljungbox
        lb_test = acorr_ljungbox(y, lags=10, return_df=True)
        print(f"   - Ljung-Box p-value (lag 10): {lb_test['lb_pvalue'].iloc[-1]:.4f}")
        
        break  # Solo analizar la primera combinación como ejemplo

# Verificar si el problema es la escala de los retornos
print("\n📈 ANÁLISIS DE ESCALA:")
print(f"   Los retornos logarítmicos típicamente son muy pequeños (~0.001 - 0.01)")
print(f"   Esto puede causar problemas en los modelos si no se escalan apropiadamente")

# Verificar los precios originales vs transformados
print("\n💡 COMPARACIÓN PRECIO ORIGINAL vs RETORNOS:")
original_prices = daily_features[TARGET_VAR].dropna()
print(f"   Precio promedio original: ${original_prices.mean():.2f}")
print(f"   Volatilidad precio original: ${original_prices.std():.2f}")
print(f"   Coef. Variación precios: {original_prices.std()/original_prices.mean():.3f}")

# Calcular retornos simples para comparar
simple_returns = original_prices.pct_change().dropna()
print(f"\n   Retorno simple promedio: {simple_returns.mean():.6f}")
print(f"   Volatilidad retorno simple: {simple_returns.std():.6f}")
print(f"   Retorno anualizado: {simple_returns.mean() * 252:.2%}")


🔍 DIAGNÓSTICO DEL PROBLEMA DE PREDICCIÓN

📊 Combinación: fundamental
   Estadísticas de y (retornos logarítmicos):
   - Media: 0.000091
   - Std: 0.042413
   - Min: -0.144173
   - Max: 0.140671
   - Mediana: 0.000042
   - Outliers: 6 (0.4%)
   - Ljung-Box p-value (lag 10): 0.0000

📈 ANÁLISIS DE ESCALA:
   Los retornos logarítmicos típicamente son muy pequeños (~0.001 - 0.01)
   Esto puede causar problemas en los modelos si no se escalan apropiadamente

💡 COMPARACIÓN PRECIO ORIGINAL vs RETORNOS:
   Precio promedio original: $496.74
   Volatilidad precio original: $29.71
   Coef. Variación precios: 0.060

   Retorno simple promedio: 0.000976
   Volatilidad retorno simple: 0.042454
   Retorno anualizado: 24.61%


In [15]:
# Crear DataFrame con resultados para análisis

results_df = pd.DataFrame(all_results)

# Eliminar columna 'full_result' para visualización
if 'full_result' in results_df.columns:
    results_df_display = results_df.drop('full_result', axis=1)
else:
    results_df_display = results_df

# Ordenar por RMSE de test
results_df_display = results_df_display.sort_values('test_rmse')

print("📊 TABLA DE RESULTADOS - TOP 10 MODELOS")
print("=" * 100)

# Mostrar top 10 modelos
print(results_df_display[['model', 'combination', 'test_rmse', 'test_mae', 'test_mape', 'test_r2', 
                          'directional_accuracy', 'hit_rate_2pct']].head(10).to_string())

# Estadísticas por tipo de modelo
print("\n\n📈 ESTADÍSTICAS POR TIPO DE MODELO")
print("=" * 60)

model_stats = results_df_display.groupby('model').agg({
    'test_rmse': ['mean', 'std', 'min'],
    'test_mape': ['mean', 'std', 'min'],
    'directional_accuracy': ['mean', 'max'],
    'hit_rate_2pct': ['mean', 'max']
}).round(4)

print(model_stats)

# Estadísticas por combinación de variables
print("\n\n📈 ESTADÍSTICAS POR COMBINACIÓN DE VARIABLES")
print("=" * 60)

combo_stats = results_df_display.groupby('combination').agg({
    'test_rmse': ['mean', 'std', 'min'],
    'test_mape': ['mean', 'std', 'min'],
    'directional_accuracy': ['mean', 'max'],
    'hit_rate_2pct': ['mean', 'max']
}).round(4)

print(combo_stats)


📊 TABLA DE RESULTADOS - TOP 10 MODELOS
           model  combination  test_rmse  test_mae     test_mape   test_r2  directional_accuracy  hit_rate_2pct
11       XGBoost       regime   0.037287  0.029024  2.319075e+12 -0.000005              1.388889            0.0
6        XGBoost      hibrida   0.037287  0.029024  2.305232e+12 -0.000005              1.388889            0.0
1        XGBoost  fundamental   0.037287  0.029024  2.252845e+12 -0.000006              1.388889            0.0
0   ARIMAX-GARCH  fundamental   0.037318  0.029067  4.805251e+11 -0.001684             40.277778            0.0
10  ARIMAX-GARCH       regime   0.037485  0.029317  3.240301e+13 -0.010670             44.444444            0.0
5   ARIMAX-GARCH      hibrida   0.037729  0.029575  4.398519e+13 -0.023842             43.055556            0.0
12      LightGBM       regime   0.038103  0.029245  1.323727e+13 -0.044276             52.777778            0.0
14         MIDAS       regime   0.040631  0.032050  4.560164e+13 

## Métricas que dan pena!!. Intentemos con otra aproximación

## 🔧 VERSIÓN 2.0: Modelos Mejorados con Diferentes Enfoques

### Cambios principales:
1. **Predecir directamente el precio** (no retornos)
2. **Usar escalamiento robusto** (RobustScaler)
3. **Incluir más features de contexto**
4. **Validación más estricta**


In [16]:
# VERSIÓN MEJORADA: Preparación de datos con diferentes enfoques

def prepare_data_v2(df, target_var, feature_vars, forecast_horizon=1, method='price'):
    """
    Versión mejorada de preparación de datos con múltiples métodos
    
    Parameters:
    -----------
    method : str
        'price': Predecir directamente el precio (con escalamiento)
        'returns': Predecir retornos simples
        'log_returns': Predecir retornos logarítmicos
        'diff': Predecir primera diferencia
    """
    
    # Seleccionar features
    X = df[feature_vars].copy()
    
    # Variable objetivo: precio en t+forecast_horizon
    y_future = df[target_var].shift(-forecast_horizon)
    
    # Eliminar NaN iniciales
    valid_idx = ~(X.isna().any(axis=1) | y_future.isna())
    X = X[valid_idx]
    y_future = y_future[valid_idx]
    y_current = df[target_var][valid_idx]
    
    # Preparar features (siempre usar cambios/ratios para estacionariedad)
    X_processed = pd.DataFrame(index=X.index)
    
    for col in X.columns:
        if col.endswith('_lag_1'):
            # Mantener lags como están
            X_processed[col] = X[col]
        elif 'volatility' in col or 'rsi' in col or 'bb_' in col:
            # Indicadores técnicos mantener como están
            X_processed[col] = X[col]
        elif col in ['VIX', 'tasa_interes_banxico', 'tiie_28_dias']:
            # Tasas y volatilidad en niveles
            X_processed[col] = X[col]
        else:
            # Para precios/commodities usar retornos
            X_processed[f'{col}_return'] = X[col].pct_change()
            X_processed[f'{col}_ma_ratio'] = X[col] / X[col].rolling(20).mean()
    
    # Preparar variable objetivo según el método
    if method == 'price':
        # Predecir directamente el precio
        y = y_future
        # Agregar el precio actual como feature importante
        X_processed['current_price'] = y_current
        X_processed['price_ma20'] = y_current.rolling(20).mean()
        X_processed['price_std20'] = y_current.rolling(20).std()
        
    elif method == 'returns':
        # Predecir retorno simple
        y = (y_future - y_current) / y_current
        
    elif method == 'log_returns':
        # Predecir retorno logarítmico
        y = np.log(y_future / y_current)
        
    elif method == 'diff':
        # Predecir primera diferencia
        y = y_future - y_current
        X_processed['current_price'] = y_current
        
    else:
        raise ValueError(f"Método no reconocido: {method}")
    
    # Eliminar filas con NaN o infinitos
    valid_final = ~(X_processed.isna().any(axis=1) | y.isna() | 
                    np.isinf(X_processed).any(axis=1) | np.isinf(y))
    
    X_clean = X_processed[valid_final]
    y_clean = y[valid_final]
    
    # Guardar información para transformación inversa
    transform_info = {
        'method': method,
        'current_prices': y_current[valid_final],
        'feature_names': list(X_clean.columns)
    }
    
    return X_clean, y_clean, transform_info

# Preparar datos con diferentes métodos
print("📦 Preparando datos con MÉTODO MEJORADO (predecir precio directamente)...\n")

prepared_data_v2 = {}
for combo_name, vars_list in variable_combinations.items():
    print(f"Procesando {combo_name}...")
    try:
        # Usar método 'price' para predecir directamente el precio
        X, y, info = prepare_data_v2(daily_features, TARGET_VAR, vars_list, method='price')
        prepared_data_v2[combo_name] = {'X': X, 'y': y, 'info': info}
        print(f"   ✅ Shape: X={X.shape}, y={y.shape}")
        print(f"   Rango de y (precios): ${y.min():.2f} - ${y.max():.2f}")
        print(f"   Media de y: ${y.mean():.2f}")
    except Exception as e:
        print(f"   ❌ Error: {e}")

print("\n✅ Datos V2 preparados")


📦 Preparando datos con MÉTODO MEJORADO (predecir precio directamente)...

Procesando fundamental...
   ✅ Shape: X=(1429, 13), y=(1429,)
   Rango de y (precios): $408.50 - $590.70
   Media de y: $497.30
Procesando hibrida...
   ✅ Shape: X=(1429, 12), y=(1429,)
   Rango de y (precios): $408.50 - $590.70
   Media de y: $497.30
Procesando regime...
   ✅ Shape: X=(1429, 13), y=(1429,)
   Rango de y (precios): $408.50 - $590.70
   Media de y: $497.30

✅ Datos V2 preparados


In [17]:
# MODELOS MEJORADOS V2: Con escalamiento y predicción directa de precios

def train_xgboost_v2(X_train, y_train, X_test, y_test, combo_name, n_trials=30):
    """
    XGBoost mejorado con escalamiento y predicción directa de precios
    """
    from sklearn.preprocessing import RobustScaler
    
    # Escalar features y target
    scaler_X = RobustScaler()
    scaler_y = RobustScaler()
    
    X_train_scaled = scaler_X.fit_transform(X_train)
    X_test_scaled = scaler_X.transform(X_test)
    
    y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).ravel()
    y_test_original = y_test.values  # Guardar valores originales para métricas
    
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 200, 1500),
            'max_depth': trial.suggest_int('max_depth', 3, 12),
            'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 15),
            'gamma': trial.suggest_float('gamma', 0, 0.5),
            'reg_alpha': trial.suggest_float('reg_alpha', 0, 2.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0, 2.0),
            'random_state': 42,
            'objective': 'reg:squarederror',
            'n_jobs': -1
        }
        
        # Validación cruzada temporal
        tscv_inner = TimeSeriesCV(n_splits=3, train_size=400, test_size=50)
        cv_scores = []
        
        for train_idx, val_idx in tscv_inner.split(X_train_scaled):
            X_cv_train = X_train_scaled[train_idx]
            y_cv_train = y_train_scaled[train_idx]
            X_cv_val = X_train_scaled[val_idx]
            y_cv_val = y_train_scaled[val_idx]
            
            model = xgb.XGBRegressor(**params)
            model.fit(X_cv_train, y_cv_train)
            
            # Predecir y desescalar
            y_pred_scaled = model.predict(X_cv_val)
            y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()
            y_true = scaler_y.inverse_transform(y_cv_val.reshape(-1, 1)).ravel()
            
            # RMSE en escala original
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))
            cv_scores.append(rmse)
        
        return np.mean(cv_scores)
    
    # Optimización con Optuna
    study = optuna.create_study(direction='minimize', study_name=f'xgboost_v2_{combo_name}')
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
    
    # Entrenar modelo final
    best_params = study.best_params
    best_params['random_state'] = 42
    best_params['objective'] = 'reg:squarederror'
    best_params['n_jobs'] = -1
    
    final_model = xgb.XGBRegressor(**best_params)
    final_model.fit(X_train_scaled, y_train_scaled)
    
    # Predicciones
    y_pred_train_scaled = final_model.predict(X_train_scaled)
    y_pred_test_scaled = final_model.predict(X_test_scaled)
    
    # Desescalar predicciones
    y_pred_train = scaler_y.inverse_transform(y_pred_train_scaled.reshape(-1, 1)).ravel()
    y_pred_test = scaler_y.inverse_transform(y_pred_test_scaled.reshape(-1, 1)).ravel()
    
    # Métricas en escala original (precios)
    train_metrics = evaluate_model(y_train.values, y_pred_train, f"XGBoost_v2_{combo_name}_train")
    test_metrics = evaluate_model(y_test_original, y_pred_test, f"XGBoost_v2_{combo_name}_test")
    
    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': final_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    return {
        'model': final_model,
        'scalers': {'X': scaler_X, 'y': scaler_y},
        'best_params': best_params,
        'train_metrics': train_metrics,
        'test_metrics': test_metrics,
        'predictions': {'train': y_pred_train, 'test': y_pred_test},
        'feature_importance': feature_importance
    }

def train_lightgbm_v2(X_train, y_train, X_test, y_test, combo_name, n_trials=30):
    """
    LightGBM mejorado con escalamiento y predicción directa de precios
    """
    from sklearn.preprocessing import RobustScaler
    
    # Escalar features y target
    scaler_X = RobustScaler()
    scaler_y = RobustScaler()
    
    X_train_scaled = scaler_X.fit_transform(X_train)
    X_test_scaled = scaler_X.transform(X_test)
    
    y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).ravel()
    y_test_original = y_test.values
    
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 200, 1500),
            'num_leaves': trial.suggest_int('num_leaves', 20, 200),
            'max_depth': trial.suggest_int('max_depth', 3, 15),
            'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True),
            'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
            'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
            'reg_alpha': trial.suggest_float('reg_alpha', 0, 2.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0, 2.0),
            'random_state': 42,
            'objective': 'regression',
            'metric': 'rmse',
            'n_jobs': -1,
            'verbosity': -1
        }
        
        # Validación cruzada
        model = lgb.LGBMRegressor(**params)
        
        # Train con early stopping
        model.fit(X_train_scaled, y_train_scaled,
                 eval_set=[(X_test_scaled, scaler_y.transform(y_test.values.reshape(-1, 1)).ravel())],
                 callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)])
        
        # Evaluar
        y_pred_scaled = model.predict(X_test_scaled)
        y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()
        rmse = np.sqrt(mean_squared_error(y_test_original, y_pred))
        
        return rmse
    
    # Optimización
    study = optuna.create_study(direction='minimize', study_name=f'lightgbm_v2_{combo_name}')
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
    
    # Modelo final
    best_params = study.best_params
    best_params.update({
        'random_state': 42,
        'objective': 'regression',
        'metric': 'rmse',
        'n_jobs': -1,
        'verbosity': -1
    })
    
    final_model = lgb.LGBMRegressor(**best_params)
    final_model.fit(X_train_scaled, y_train_scaled)
    
    # Predicciones
    y_pred_train_scaled = final_model.predict(X_train_scaled)
    y_pred_test_scaled = final_model.predict(X_test_scaled)
    
    # Desescalar
    y_pred_train = scaler_y.inverse_transform(y_pred_train_scaled.reshape(-1, 1)).ravel()
    y_pred_test = scaler_y.inverse_transform(y_pred_test_scaled.reshape(-1, 1)).ravel()
    
    # Métricas
    train_metrics = evaluate_model(y_train.values, y_pred_train, f"LightGBM_v2_{combo_name}_train")
    test_metrics = evaluate_model(y_test_original, y_pred_test, f"LightGBM_v2_{combo_name}_test")
    
    return {
        'model': final_model,
        'scalers': {'X': scaler_X, 'y': scaler_y},
        'best_params': best_params,
        'train_metrics': train_metrics,
        'test_metrics': test_metrics,
        'predictions': {'train': y_pred_train, 'test': y_pred_test}
    }

print("✅ Modelos V2 definidos con escalamiento robusto")


✅ Modelos V2 definidos con escalamiento robusto


In [18]:
# MODELOS V2 AVANZADOS - SERIES DE TIEMPO ESPECIALIZADOS

def train_arimax_garch_v2(X_train, y_train, X_test, y_test, combo_name, n_trials=15):
    """
    ARIMAX-GARCH V2: Predicción directa de precios con primera diferencia
    - Usa primera diferencia en lugar de retornos log
    - GARCH sobre residuos en escala de precios
    - Mejor manejo de la volatilidad
    """
    from arch import arch_model
    from statsmodels.tsa.arima.model import ARIMA
    from sklearn.preprocessing import StandardScaler
    import warnings
    warnings.filterwarnings('ignore')
    
    print(f"   🔄 Optimizando ARIMAX-GARCH V2 para {combo_name}...")
    
    # Preparar datos - usar primera diferencia
    y_train_diff = y_train.diff().dropna()
    y_test_diff = y_test.diff().dropna()
    
    # Ajustar X para coincidir con las diferencias
    X_train_adj = X_train.iloc[1:].copy()  # Quitar primera observación
    X_test_adj = X_test.iloc[1:].copy()
    
    # Escalar variables exógenas
    scaler_X = StandardScaler()
    X_train_scaled = scaler_X.fit_transform(X_train_adj)
    X_test_scaled = scaler_X.transform(X_test_adj)
    
    def objective(trial):
        try:
            # Parámetros ARIMAX
            p = trial.suggest_int('p', 0, 3)
            d = 0  # Ya diferenciamos manualmente
            q = trial.suggest_int('q', 0, 3)
            
            # Parámetros GARCH
            garch_p = trial.suggest_int('garch_p', 1, 2)
            garch_q = trial.suggest_int('garch_q', 1, 2)
            
            # Entrenar ARIMAX
            arimax_model = ARIMA(
                y_train_diff,
                exog=X_train_scaled,
                order=(p, d, q)
            )
            arimax_fit = arimax_model.fit(method_kwargs={'warn_convergence': False})
            
            # Obtener residuos
            residuals = arimax_fit.resid
            
            # Entrenar GARCH sobre residuos
            garch_model = arch_model(
                residuals,
                vol='GARCH',
                p=garch_p,
                q=garch_q,
                rescale=False
            )
            garch_fit = garch_model.fit(disp='off', show_warning=False)
            
            # Predicciones ARIMAX
            arimax_pred = arimax_fit.forecast(steps=len(y_test_diff), exog=X_test_scaled)
            
            # Convertir diferencias a niveles
            y_pred_levels = np.zeros(len(y_test))
            y_pred_levels[0] = y_train.iloc[-1]  # Último valor conocido
            
            for i in range(1, len(y_pred_levels)):
                y_pred_levels[i] = y_pred_levels[i-1] + arimax_pred.iloc[i-1]
            
            # Calcular RMSE
            rmse = np.sqrt(mean_squared_error(y_test.values, y_pred_levels))
            return rmse
            
        except Exception as e:
            return float('inf')
    
    # Optimización
    study = optuna.create_study(direction='minimize', study_name=f'arimax_garch_v2_{combo_name}')
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
    
    # Modelo final con mejores parámetros
    best_params = study.best_params
    
    try:
        # ARIMAX final
        final_arimax = ARIMA(
            y_train_diff,
            exog=X_train_scaled,
            order=(best_params['p'], 0, best_params['q'])
        )
        arimax_fit = final_arimax.fit(method_kwargs={'warn_convergence': False})
        
        # GARCH final
        residuals = arimax_fit.resid
        final_garch = arch_model(
            residuals,
            vol='GARCH',
            p=best_params['garch_p'],
            q=best_params['garch_q'],
            rescale=False
        )
        garch_fit = final_garch.fit(disp='off', show_warning=False)
        
        # Predicciones finales
        arimax_pred_train = arimax_fit.fittedvalues
        arimax_pred_test = arimax_fit.forecast(steps=len(y_test_diff), exog=X_test_scaled)
        
        # Convertir a niveles de precios
        y_pred_train_levels = np.zeros(len(y_train))
        y_pred_train_levels[0] = y_train.iloc[0]
        for i in range(1, len(y_pred_train_levels)):
            y_pred_train_levels[i] = y_pred_train_levels[i-1] + arimax_pred_train.iloc[i-1]
        
        y_pred_test_levels = np.zeros(len(y_test))
        y_pred_test_levels[0] = y_train.iloc[-1]
        for i in range(1, len(y_pred_test_levels)):
            y_pred_test_levels[i] = y_pred_test_levels[i-1] + arimax_pred_test.iloc[i-1]
        
        # Métricas
        train_metrics = evaluate_model(y_train.values, y_pred_train_levels, f"ARIMAX_GARCH_v2_{combo_name}_train")
        test_metrics = evaluate_model(y_test.values, y_pred_test_levels, f"ARIMAX_GARCH_v2_{combo_name}_test")
        
        return {
            'arimax_model': arimax_fit,
            'garch_model': garch_fit,
            'scaler_X': scaler_X,
            'best_params': best_params,
            'train_metrics': train_metrics,
            'test_metrics': test_metrics,
            'predictions': {'train': y_pred_train_levels, 'test': y_pred_test_levels}
        }
        
    except Exception as e:
        print(f"   ❌ Error en modelo final: {e}")
        return None


def train_markov_regime_v2(X_train, y_train, X_test, y_test, combo_name, n_trials=15):
    """
    Markov Regime-Switching V2: Detección mejorada de regímenes
    - Regímenes basados en niveles de precio y volatilidad
    - Modelos separados por régimen con escalamiento
    - Mejor detección de cambios estructurales
    """
    from sklearn.mixture import GaussianMixture
    from sklearn.preprocessing import StandardScaler
    import warnings
    warnings.filterwarnings('ignore')
    
    print(f"   🔄 Optimizando Markov Regime V2 para {combo_name}...")
    
    # Calcular características para detección de regímenes
    price_ma_20 = y_train.rolling(20).mean()
    price_volatility = y_train.rolling(20).std()
    price_level = (y_train - y_train.rolling(60).mean()) / y_train.rolling(60).std()
    
    # Crear features para regímenes (sin NaN)
    regime_features = pd.DataFrame({
        'price_level': price_level,
        'volatility': price_volatility,
        'ma_ratio': y_train / price_ma_20
    }).dropna()
    
    # Ajustar índices
    start_idx = regime_features.index[0]
    y_train_clean = y_train.loc[start_idx:]
    X_train_clean = X_train.loc[start_idx:]
    
    def objective(trial):
        try:
            # Parámetros
            n_regimes = trial.suggest_int('n_regimes', 2, 4)
            
            # Detectar regímenes con Gaussian Mixture
            gmm = GaussianMixture(
                n_components=n_regimes,
                covariance_type='full',
                random_state=42,
                max_iter=100
            )
            
            regime_labels = gmm.fit_predict(regime_features.values)
            
            # Entrenar modelos por régimen
            regime_models = {}
            regime_scalers = {}
            
            for regime in range(n_regimes):
                regime_mask = regime_labels == regime
                
                if np.sum(regime_mask) < 10:  # Muy pocas observaciones
                    continue
                
                # Datos del régimen
                X_regime = X_train_clean[regime_mask]
                y_regime = y_train_clean[regime_mask]
                
                if len(X_regime) < 5:
                    continue
                
                # Escalar datos del régimen
                scaler_X_regime = StandardScaler()
                scaler_y_regime = StandardScaler()
                
                X_regime_scaled = scaler_X_regime.fit_transform(X_regime)
                y_regime_scaled = scaler_y_regime.fit_transform(y_regime.values.reshape(-1, 1)).ravel()
                
                # Modelo simple para el régimen
                from sklearn.ensemble import RandomForestRegressor
                model_regime = RandomForestRegressor(
                    n_estimators=50,
                    max_depth=trial.suggest_int(f'max_depth_{regime}', 3, 10),
                    random_state=42,
                    n_jobs=-1
                )
                
                model_regime.fit(X_regime_scaled, y_regime_scaled)
                
                regime_models[regime] = model_regime
                regime_scalers[regime] = {'X': scaler_X_regime, 'y': scaler_y_regime}
            
            if len(regime_models) == 0:
                return float('inf')
            
            # Predicciones en test
            # Detectar regímenes en test
            price_ma_20_test = pd.concat([y_train.tail(20), y_test]).rolling(20).mean().iloc[20:]
            price_volatility_test = pd.concat([y_train.tail(20), y_test]).rolling(20).std().iloc[20:]
            price_level_test = (y_test - pd.concat([y_train.tail(60), y_test]).rolling(60).mean().iloc[60:]) / pd.concat([y_train.tail(60), y_test]).rolling(60).std().iloc[60:]
            
            regime_features_test = pd.DataFrame({
                'price_level': price_level_test,
                'volatility': price_volatility_test,
                'ma_ratio': y_test / price_ma_20_test
            }).fillna(0)  # Llenar NaN con 0
            
            test_regime_labels = gmm.predict(regime_features_test.values)
            
            # Predicciones por régimen
            y_pred_test = np.zeros(len(y_test))
            
            for i, regime in enumerate(test_regime_labels):
                if regime in regime_models:
                    X_test_point = X_test.iloc[i:i+1]
                    X_test_scaled = regime_scalers[regime]['X'].transform(X_test_point)
                    y_pred_scaled = regime_models[regime].predict(X_test_scaled)
                    y_pred_test[i] = regime_scalers[regime]['y'].inverse_transform(y_pred_scaled.reshape(-1, 1))[0, 0]
                else:
                    # Usar régimen más común si no existe
                    common_regime = max(regime_models.keys())
                    X_test_point = X_test.iloc[i:i+1]
                    X_test_scaled = regime_scalers[common_regime]['X'].transform(X_test_point)
                    y_pred_scaled = regime_models[common_regime].predict(X_test_scaled)
                    y_pred_test[i] = regime_scalers[common_regime]['y'].inverse_transform(y_pred_scaled.reshape(-1, 1))[0, 0]
            
            rmse = np.sqrt(mean_squared_error(y_test.values, y_pred_test))
            return rmse
            
        except Exception as e:
            return float('inf')
    
    # Optimización
    study = optuna.create_study(direction='minimize', study_name=f'markov_regime_v2_{combo_name}')
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
    
    # Modelo final
    best_params = study.best_params
    n_regimes = best_params['n_regimes']
    
    try:
        # GMM final
        final_gmm = GaussianMixture(
            n_components=n_regimes,
            covariance_type='full',
            random_state=42,
            max_iter=100
        )
        
        regime_labels = final_gmm.fit_predict(regime_features.values)
        
        # Entrenar modelos finales por régimen
        final_regime_models = {}
        final_regime_scalers = {}
        
        for regime in range(n_regimes):
            regime_mask = regime_labels == regime
            
            if np.sum(regime_mask) < 10:
                continue
            
            X_regime = X_train_clean[regime_mask]
            y_regime = y_train_clean[regime_mask]
            
            if len(X_regime) < 5:
                continue
            
            scaler_X_regime = StandardScaler()
            scaler_y_regime = StandardScaler()
            
            X_regime_scaled = scaler_X_regime.fit_transform(X_regime)
            y_regime_scaled = scaler_y_regime.fit_transform(y_regime.values.reshape(-1, 1)).ravel()
            
            from sklearn.ensemble import RandomForestRegressor
            model_regime = RandomForestRegressor(
                n_estimators=100,
                max_depth=best_params.get(f'max_depth_{regime}', 6),
                random_state=42,
                n_jobs=-1
            )
            
            model_regime.fit(X_regime_scaled, y_regime_scaled)
            
            final_regime_models[regime] = model_regime
            final_regime_scalers[regime] = {'X': scaler_X_regime, 'y': scaler_y_regime}
        
        # Predicciones finales (train y test)
        # Train predictions
        y_pred_train = np.zeros(len(y_train_clean))
        for i, regime in enumerate(regime_labels):
            if regime in final_regime_models:
                X_train_point = X_train_clean.iloc[i:i+1]
                X_train_scaled = final_regime_scalers[regime]['X'].transform(X_train_point)
                y_pred_scaled = final_regime_models[regime].predict(X_train_scaled)
                y_pred_train[i] = final_regime_scalers[regime]['y'].inverse_transform(y_pred_scaled.reshape(-1, 1))[0, 0]
        
        # Test predictions (como en objective)
        price_ma_20_test = pd.concat([y_train.tail(20), y_test]).rolling(20).mean().iloc[20:]
        price_volatility_test = pd.concat([y_train.tail(20), y_test]).rolling(20).std().iloc[20:]
        price_level_test = (y_test - pd.concat([y_train.tail(60), y_test]).rolling(60).mean().iloc[60:]) / pd.concat([y_train.tail(60), y_test]).rolling(60).std().iloc[60:]
        
        regime_features_test = pd.DataFrame({
            'price_level': price_level_test,
            'volatility': price_volatility_test,
            'ma_ratio': y_test / price_ma_20_test
        }).fillna(0)
        
        test_regime_labels = final_gmm.predict(regime_features_test.values)
        
        y_pred_test = np.zeros(len(y_test))
        for i, regime in enumerate(test_regime_labels):
            if regime in final_regime_models:
                X_test_point = X_test.iloc[i:i+1]
                X_test_scaled = final_regime_scalers[regime]['X'].transform(X_test_point)
                y_pred_scaled = final_regime_models[regime].predict(X_test_scaled)
                y_pred_test[i] = final_regime_scalers[regime]['y'].inverse_transform(y_pred_scaled.reshape(-1, 1))[0, 0]
            else:
                common_regime = max(final_regime_models.keys())
                X_test_point = X_test.iloc[i:i+1]
                X_test_scaled = final_regime_scalers[common_regime]['X'].transform(X_test_point)
                y_pred_scaled = final_regime_models[common_regime].predict(X_test_scaled)
                y_pred_test[i] = final_regime_scalers[common_regime]['y'].inverse_transform(y_pred_scaled.reshape(-1, 1))[0, 0]
        
        # Métricas
        train_metrics = evaluate_model(y_train_clean.values, y_pred_train, f"Markov_Regime_v2_{combo_name}_train")
        test_metrics = evaluate_model(y_test.values, y_pred_test, f"Markov_Regime_v2_{combo_name}_test")
        
        return {
            'gmm_model': final_gmm,
            'regime_models': final_regime_models,
            'regime_scalers': final_regime_scalers,
            'regime_features': regime_features,
            'best_params': best_params,
            'train_metrics': train_metrics,
            'test_metrics': test_metrics,
            'predictions': {'train': y_pred_train, 'test': y_pred_test}
        }
        
    except Exception as e:
        print(f"   ❌ Error en modelo final: {e}")
        return None


def train_midas_v2(X_train, y_train, X_test, y_test, combo_name, n_trials=15):
    """
    MIDAS V2: Optimizado para predicción directa de precios
    - Variables mensuales usadas directamente
    - Mejor manejo de frecuencias mixtas
    - Escalamiento robusto con validación de datos
    """
    from sklearn.preprocessing import StandardScaler, RobustScaler
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.linear_model import Ridge
    import warnings
    warnings.filterwarnings('ignore')
    
    print(f"   🔄 Optimizando MIDAS V2 para {combo_name}...")
    
    # Identificar variables mensuales vs diarias
    monthly_vars = []
    daily_vars = []
    
    for col in X_train.columns:
        # Variables que típicamente son mensuales
        if any(keyword in col.lower() for keyword in ['pmi', 'cpi', 'gdp', 'employment', 'retail', 'industrial']):
            monthly_vars.append(col)
        else:
            daily_vars.append(col)
    
    print(f"   📊 Variables mensuales detectadas: {len(monthly_vars)}")
    print(f"   📊 Variables diarias detectadas: {len(daily_vars)}")
    
    def create_midas_features(X, y, lookback_months=6):
        """Crear features MIDAS con diferentes horizontes temporales y validación robusta"""
        midas_features = []
        
        # Features diarias (lags cortos)
        for col in daily_vars:
            if col in X.columns:
                for lag in [1, 5, 10, 20]:
                    feature = X[col].shift(lag)
                    # Validar que no tenga valores infinitos o extremos
                    if not feature.isnull().all():
                        # Reemplazar infinitos con NaN
                        feature = feature.replace([np.inf, -np.inf], np.nan)
                        # Winsorizar valores extremos (percentiles 1 y 99)
                        if not feature.isnull().all():
                            p1, p99 = feature.quantile([0.01, 0.99])
                            feature = feature.clip(lower=p1, upper=p99)
                        midas_features.append(feature)
        
        # Features mensuales (lags más largos, agregaciones)
        for col in monthly_vars:
            if col in X.columns:
                # Lags mensuales (aproximadamente 22 días hábiles por mes)
                for month_lag in range(1, lookback_months + 1):
                    lag_days = month_lag * 22
                    if lag_days < len(X):
                        feature = X[col].shift(lag_days)
                        # Validar y limpiar
                        if not feature.isnull().all():
                            feature = feature.replace([np.inf, -np.inf], np.nan)
                            if not feature.isnull().all():
                                p1, p99 = feature.quantile([0.01, 0.99])
                                feature = feature.clip(lower=p1, upper=p99)
                            midas_features.append(feature)
                
                # Promedios móviles mensuales
                for window in [22, 66]:  # 1 mes, 3 meses
                    if window < len(X):
                        feature = X[col].rolling(window, min_periods=window//2).mean()
                        if not feature.isnull().all():
                            feature = feature.replace([np.inf, -np.inf], np.nan)
                            if not feature.isnull().all():
                                p1, p99 = feature.quantile([0.01, 0.99])
                                feature = feature.clip(lower=p1, upper=p99)
                            midas_features.append(feature)
        
        # Features de precio (autorregresivos)
        for lag in [1, 5, 10, 20, 60]:
            if lag < len(y):
                feature = y.shift(lag)
                if not feature.isnull().all():
                    feature = feature.replace([np.inf, -np.inf], np.nan)
                    if not feature.isnull().all():
                        p1, p99 = feature.quantile([0.01, 0.99])
                        feature = feature.clip(lower=p1, upper=p99)
                    midas_features.append(feature)
        
        # Ratios y diferencias (con validación)
        if len(daily_vars) > 0 and daily_vars[0] in X.columns:
            denominator = X[daily_vars[0]]
            # Evitar división por cero
            denominator_safe = denominator.replace(0, np.nan)
            if not denominator_safe.isnull().all():
                ratio = y / denominator_safe
                ratio = ratio.replace([np.inf, -np.inf], np.nan)
                if not ratio.isnull().all():
                    p1, p99 = ratio.quantile([0.01, 0.99])
                    ratio = ratio.clip(lower=p1, upper=p99)
                midas_features.append(ratio)
        
        # Combinar todas las features
        if len(midas_features) == 0:
            print("   ⚠️ No se pudieron crear features MIDAS válidas")
            return pd.DataFrame()
        
        midas_df = pd.concat(midas_features, axis=1)
        midas_df.columns = [f'midas_feature_{i}' for i in range(len(midas_features))]
        
        # Eliminar columnas que son completamente NaN
        midas_df = midas_df.dropna(axis=1, how='all')
        
        # Eliminar filas con demasiados NaN (más del 50%)
        threshold = len(midas_df.columns) * 0.5
        midas_df = midas_df.dropna(thresh=threshold)
        
        # Llenar NaN restantes con forward fill y backward fill
        midas_df = midas_df.fillna(method='ffill').fillna(method='bfill')
        
        # Verificación final de valores infinitos o extremos
        for col in midas_df.columns:
            if midas_df[col].isnull().all():
                continue
            # Reemplazar cualquier infinito restante
            midas_df[col] = midas_df[col].replace([np.inf, -np.inf], np.nan)
            # Llenar NaN con la mediana
            if midas_df[col].isnull().any():
                midas_df[col] = midas_df[col].fillna(midas_df[col].median())
        
        return midas_df
    
    # Crear features MIDAS
    midas_train = create_midas_features(X_train, y_train)
    midas_test = create_midas_features(X_test, y_test)
    
    if midas_train.empty or midas_test.empty:
        print(f"   ❌ No se pudieron crear features MIDAS válidas")
        return None
    
    # Asegurar que ambos conjuntos tengan las mismas columnas
    common_cols = midas_train.columns.intersection(midas_test.columns)
    if len(common_cols) == 0:
        print(f"   ❌ No hay columnas comunes entre train y test")
        return None
    
    midas_train = midas_train[common_cols]
    midas_test = midas_test[common_cols]
    
    # Ajustar y para coincidir con features MIDAS
    y_train_adj = y_train.loc[midas_train.index]
    y_test_adj = y_test.loc[midas_test.index]
    
    print(f"   📊 Features MIDAS creadas: {midas_train.shape[1]}")
    print(f"   📊 Observaciones train: {len(midas_train)}")
    print(f"   📊 Observaciones test: {len(midas_test)}")
    
    # Verificación final de datos
    if np.any(np.isinf(midas_train.values)) or np.any(np.isinf(midas_test.values)):
        print(f"   ❌ Aún hay valores infinitos en los datos")
        return None
    
    if np.any(np.isnan(midas_train.values)) or np.any(np.isnan(midas_test.values)):
        print(f"   ❌ Aún hay valores NaN en los datos")
        return None
    
    def objective(trial):
        try:
            # Seleccionar tipo de modelo
            model_type = trial.suggest_categorical('model_type', ['ridge', 'random_forest'])
            
            # Usar RobustScaler en lugar de StandardScaler para mejor manejo de outliers
            scaler_X = RobustScaler()
            scaler_y = RobustScaler()
            
            X_train_scaled = scaler_X.fit_transform(midas_train)
            y_train_scaled = scaler_y.fit_transform(y_train_adj.values.reshape(-1, 1)).ravel()
            
            X_test_scaled = scaler_X.transform(midas_test)
            
            # Verificar que no hay infinitos después del escalamiento
            if np.any(np.isinf(X_train_scaled)) or np.any(np.isinf(X_test_scaled)):
                return float('inf')
            
            if model_type == 'ridge':
                alpha = trial.suggest_float('alpha', 0.01, 100.0, log=True)
                model = Ridge(alpha=alpha, random_state=42)
            else:  # random_forest
                n_estimators = trial.suggest_int('n_estimators', 50, 200)
                max_depth = trial.suggest_int('max_depth', 3, 15)
                min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
                
                model = RandomForestRegressor(
                    n_estimators=n_estimators,
                    max_depth=max_depth,
                    min_samples_split=min_samples_split,
                    random_state=42,
                    n_jobs=-1
                )
            
            # Entrenar
            model.fit(X_train_scaled, y_train_scaled)
            
            # Predicciones
            y_pred_scaled = model.predict(X_test_scaled)
            y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()
            
            # Verificar que las predicciones son válidas
            if np.any(np.isinf(y_pred)) or np.any(np.isnan(y_pred)):
                return float('inf')
            
            rmse = np.sqrt(mean_squared_error(y_test_adj.values, y_pred))
            return rmse
            
        except Exception as e:
            return float('inf')
    
    # Optimización
    study = optuna.create_study(direction='minimize', study_name=f'midas_v2_{combo_name}')
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
    
    # Modelo final
    best_params = study.best_params
    
    try:
        # Escaladores finales
        final_scaler_X = RobustScaler()
        final_scaler_y = RobustScaler()
        
        X_train_scaled = final_scaler_X.fit_transform(midas_train)
        y_train_scaled = final_scaler_y.fit_transform(y_train_adj.values.reshape(-1, 1)).ravel()
        
        X_test_scaled = final_scaler_X.transform(midas_test)
        
        # Verificar escalamiento final
        if np.any(np.isinf(X_train_scaled)) or np.any(np.isinf(X_test_scaled)):
            print(f"   ❌ Valores infinitos después del escalamiento final")
            return None
        
        # Modelo final
        if best_params['model_type'] == 'ridge':
            final_model = Ridge(alpha=best_params['alpha'], random_state=42)
        else:
            final_model = RandomForestRegressor(
                n_estimators=best_params['n_estimators'],
                max_depth=best_params['max_depth'],
                min_samples_split=best_params['min_samples_split'],
                random_state=42,
                n_jobs=-1
            )
        
        final_model.fit(X_train_scaled, y_train_scaled)
        
        # Predicciones finales
        y_pred_train_scaled = final_model.predict(X_train_scaled)
        y_pred_test_scaled = final_model.predict(X_test_scaled)
        
        y_pred_train = final_scaler_y.inverse_transform(y_pred_train_scaled.reshape(-1, 1)).ravel()
        y_pred_test = final_scaler_y.inverse_transform(y_pred_test_scaled.reshape(-1, 1)).ravel()
        
        # Verificar predicciones finales
        if np.any(np.isinf(y_pred_train)) or np.any(np.isinf(y_pred_test)):
            print(f"   ❌ Predicciones finales contienen infinitos")
            return None
        
        # Métricas
        train_metrics = evaluate_model(y_train_adj.values, y_pred_train, f"MIDAS_v2_{combo_name}_train")
        test_metrics = evaluate_model(y_test_adj.values, y_pred_test, f"MIDAS_v2_{combo_name}_test")
        
        return {
            'model': final_model,
            'scalers': {'X': final_scaler_X, 'y': final_scaler_y},
            'midas_features': {'train': midas_train, 'test': midas_test},
            'best_params': best_params,
            'train_metrics': train_metrics,
            'test_metrics': test_metrics,
            'predictions': {'train': y_pred_train, 'test': y_pred_test}
        }
        
    except Exception as e:
        print(f"   ❌ Error en modelo final: {e}")
        return None

print("✅ Modelos V2 avanzados de series de tiempo definidos")


✅ Modelos V2 avanzados de series de tiempo definidos


In [19]:
# EJECUTAR MODELOS V2 MEJORADOS

print("🚀 EJECUTANDO MODELOS V2 - PREDICCIÓN DIRECTA DE PRECIOS")
print("=" * 60)

results_v2 = []

# Ejecutar las tres mejores combinaciones
combinations_to_test = ['hibrida', 'fundamental', 'regime']

for combo in combinations_to_test:
    if combo in prepared_data_v2:
        print(f"\n{'='*20} COMBINACIÓN: {combo.upper()} {'='*20}")
        
        X = prepared_data_v2[combo]['X']
        y = prepared_data_v2[combo]['y']
        
        # Split 80/20
        X_train, X_test, y_train, y_test = split_time_series_data(X, y, test_size=0.2)
        
        print(f"\n📊 Datos para entrenamiento:")
        print(f"   X_train: {X_train.shape}")
        print(f"   X_test: {X_test.shape}")
        print(f"   Rango precios train: ${y_train.min():.2f} - ${y_train.max():.2f}")
        print(f"   Rango precios test: ${y_test.min():.2f} - ${y_test.max():.2f}")
        
        # 1. Probar XGBoost V2
        print(f"\n🤖 Entrenando XGBoost V2 - {combo}...")
        try:
            result_xgb = train_xgboost_v2(X_train, y_train, X_test, y_test, combo, n_trials=15)
            
            print(f"   ✅ RMSE Test: ${result_xgb['test_metrics']['rmse']:.2f}")
            print(f"   ✅ MAE Test: ${result_xgb['test_metrics']['mae']:.2f}")
            print(f"   ✅ MAPE Test: {result_xgb['test_metrics']['mape']:.1f}%")
            print(f"   ✅ R² Test: {result_xgb['test_metrics']['r2']:.3f}")
            print(f"   ✅ Dir. Accuracy: {result_xgb['test_metrics']['directional_accuracy']:.1f}%")
            print(f"   ✅ Hit Rate ±2%: {result_xgb['test_metrics']['hit_rate_2pct']:.1f}%")
            
            results_v2.append({
                'model': 'XGBoost_V2',
                'combination': combo,
                **result_xgb['test_metrics']
            })
            
            # Mostrar top features
            print(f"\n   📊 Top 5 Features más importantes:")
            for idx, row in result_xgb['feature_importance'].head(5).iterrows():
                print(f"      {row['feature']}: {row['importance']:.3f}")
                
        except Exception as e:
            print(f"   ❌ Error XGBoost: {str(e)[:100]}")
        
        # 2. Probar LightGBM V2
        print(f"\n🤖 Entrenando LightGBM V2 - {combo}...")
        try:
            result_lgb = train_lightgbm_v2(X_train, y_train, X_test, y_test, combo, n_trials=15)
            
            print(f"   ✅ RMSE Test: ${result_lgb['test_metrics']['rmse']:.2f}")
            print(f"   ✅ MAE Test: ${result_lgb['test_metrics']['mae']:.2f}")
            print(f"   ✅ MAPE Test: {result_lgb['test_metrics']['mape']:.1f}%")
            print(f"   ✅ R² Test: {result_lgb['test_metrics']['r2']:.3f}")
            print(f"   ✅ Dir. Accuracy: {result_lgb['test_metrics']['directional_accuracy']:.1f}%")
            print(f"   ✅ Hit Rate ±2%: {result_lgb['test_metrics']['hit_rate_2pct']:.1f}%")
            
            results_v2.append({
                'model': 'LightGBM_V2',
                'combination': combo,
                **result_lgb['test_metrics']
            })
            
        except Exception as e:
            print(f"   ❌ Error LightGBM: {str(e)[:100]}")
        
        # 3. Probar ARIMAX-GARCH V2
        print(f"\n🤖 Entrenando ARIMAX-GARCH V2 - {combo}...")
        try:
            result_arimax = train_arimax_garch_v2(X_train, y_train, X_test, y_test, combo, n_trials=15)
            
            if result_arimax is not None:
                print(f"   ✅ RMSE Test: ${result_arimax['test_metrics']['rmse']:.2f}")
                print(f"   ✅ MAE Test: ${result_arimax['test_metrics']['mae']:.2f}")
                print(f"   ✅ MAPE Test: {result_arimax['test_metrics']['mape']:.1f}%")
                print(f"   ✅ R² Test: {result_arimax['test_metrics']['r2']:.3f}")
                print(f"   ✅ Dir. Accuracy: {result_arimax['test_metrics']['directional_accuracy']:.1f}%")
                print(f"   ✅ Hit Rate ±2%: {result_arimax['test_metrics']['hit_rate_2pct']:.1f}%")
                
                results_v2.append({
                    'model': 'ARIMAX_GARCH_V2',
                    'combination': combo,
                    **result_arimax['test_metrics']
                })
            else:
                print(f"   ❌ ARIMAX-GARCH V2 falló para {combo}")
                
        except Exception as e:
            print(f"   ❌ Error ARIMAX-GARCH: {str(e)[:100]}")
        
        # 4. Probar Markov Regime V2
        print(f"\n🤖 Entrenando Markov Regime V2 - {combo}...")
        try:
            result_markov = train_markov_regime_v2(X_train, y_train, X_test, y_test, combo, n_trials=15)
            
            if result_markov is not None:
                print(f"   ✅ RMSE Test: ${result_markov['test_metrics']['rmse']:.2f}")
                print(f"   ✅ MAE Test: ${result_markov['test_metrics']['mae']:.2f}")
                print(f"   ✅ MAPE Test: {result_markov['test_metrics']['mape']:.1f}%")
                print(f"   ✅ R² Test: {result_markov['test_metrics']['r2']:.3f}")
                print(f"   ✅ Dir. Accuracy: {result_markov['test_metrics']['directional_accuracy']:.1f}%")
                print(f"   ✅ Hit Rate ±2%: {result_markov['test_metrics']['hit_rate_2pct']:.1f}%")
                
                results_v2.append({
                    'model': 'Markov_Regime_V2',
                    'combination': combo,
                    **result_markov['test_metrics']
                })
            else:
                print(f"   ❌ Markov Regime V2 falló para {combo}")
                
        except Exception as e:
            print(f"   ❌ Error Markov Regime: {str(e)[:100]}")
        
        # 5. Probar MIDAS V2
        print(f"\n🤖 Entrenando MIDAS V2 - {combo}...")
        try:
            result_midas = train_midas_v2(X_train, y_train, X_test, y_test, combo, n_trials=15)
            
            if result_midas is not None:
                print(f"   ✅ RMSE Test: ${result_midas['test_metrics']['rmse']:.2f}")
                print(f"   ✅ MAE Test: ${result_midas['test_metrics']['mae']:.2f}")
                print(f"   ✅ MAPE Test: {result_midas['test_metrics']['mape']:.1f}%")
                print(f"   ✅ R² Test: {result_midas['test_metrics']['r2']:.3f}")
                print(f"   ✅ Dir. Accuracy: {result_midas['test_metrics']['directional_accuracy']:.1f}%")
                print(f"   ✅ Hit Rate ±2%: {result_midas['test_metrics']['hit_rate_2pct']:.1f}%")
                
                results_v2.append({
                    'model': 'MIDAS_V2',
                    'combination': combo,
                    **result_midas['test_metrics']
                })
            else:
                print(f"   ❌ MIDAS V2 falló para {combo}")
                
        except Exception as e:
            print(f"   ❌ Error MIDAS: {str(e)[:100]}")

# Comparar resultados V1 vs V2
print("\n" + "=" * 60)
print("📊 COMPARACIÓN V1 (retornos) vs V2 (precios directos)")
print("=" * 60)

if results_v2:
    df_v2 = pd.DataFrame(results_v2)
    print("\nResultados V2 (Nuevos):")
    print(df_v2[['model', 'combination', 'rmse', 'mae', 'mape', 'r2', 'directional_accuracy', 'hit_rate_2pct']])
    
    print("\n🎯 Mejora respecto a V1:")
    # Buscar mejor resultado V1 para comparar
    best_v1_rmse = results_df_display['test_rmse'].min() if 'results_df_display' in locals() else 0.017
    best_v2_rmse = df_v2['rmse'].min()
    
    if best_v1_rmse > 0:
        # Nota: V1 usa retornos (escala ~0.017), V2 usa precios (escala ~2.0)
        # Para comparar, necesitamos normalizar
        print(f"   Mejor RMSE V1 (retornos): {best_v1_rmse:.4f}")
        print(f"   Mejor RMSE V2 (precios): ${best_v2_rmse:.2f}")
        
        # Calcular RMSE relativo (% del precio promedio)
        precio_promedio = y_test.mean()
        rmse_relativo_v2 = (best_v2_rmse / precio_promedio) * 100
        print(f"   RMSE relativo V2: {rmse_relativo_v2:.2f}% del precio promedio")
        
    # Mostrar ranking por combinación
    print("\n🏆 RANKING POR COMBINACIÓN (RMSE):")
    ranking_combo = df_v2.groupby('combination')['rmse'].min().sort_values()
    for i, (combo, rmse) in enumerate(ranking_combo.items(), 1):
        print(f"   {i}. {combo}: ${rmse:.2f}")
        
    # Mostrar ranking por modelo
    print("\n🏆 RANKING POR MODELO (RMSE):")
    ranking_model = df_v2.groupby('model')['rmse'].min().sort_values()
    for i, (model, rmse) in enumerate(ranking_model.items(), 1):
        print(f"   {i}. {model}: ${rmse:.2f}")


🚀 EJECUTANDO MODELOS V2 - PREDICCIÓN DIRECTA DE PRECIOS

==================== COMBINACIÓN: HIBRIDA ====================

📊 Datos para entrenamiento:
   X_train: (1143, 12)
   X_test: (286, 12)
   Rango precios train: $408.50 - $555.99
   Rango precios test: $490.94 - $590.70

🤖 Entrenando XGBoost V2 - hibrida...


  0%|          | 0/15 [00:00<?, ?it/s]

   ✅ RMSE Test: $25.06
   ✅ MAE Test: $21.06
   ✅ MAPE Test: 3.9%
   ✅ R² Test: -1.288
   ✅ Dir. Accuracy: 46.0%
   ✅ Hit Rate ±2%: 24.1%

   📊 Top 5 Features más importantes:
      price_ma20: 0.709
      current_price: 0.038
      coking_ma_ratio: 0.031
      commodities_ma_ratio: 0.028
      VIX: 0.027

🤖 Entrenando LightGBM V2 - hibrida...


  0%|          | 0/15 [00:00<?, ?it/s]

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[78]	valid_0's rmse: 0.619285
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[976]	valid_0's rmse: 0.665067
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[98]	valid_0's rmse: 0.622412
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[92]	valid_0's rmse: 0.654818
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[83]	valid_0's rmse: 0.660847
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[796]	valid_0's rmse: 0.949103
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[32]	valid_0's rmse: 0.66119
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[49]	val

...

   ✅ RMSE Test: $179.51
   ✅ MAE Test: $149.36
   ✅ MAPE Test: 27.8%
   ✅ R² Test: -116.428
   ✅ Dir. Accuracy: 69.8%
   ✅ Hit Rate ±2%: 1.7%

🤖 Entrenando Markov Regime V2 - regime...
   🔄 Optimizando Markov Regime V2 para regime...


  0%|          | 0/15 [00:00<?, ?it/s]

   ✅ RMSE Test: $31.52
   ✅ MAE Test: $26.18
   ✅ MAPE Test: 4.8%
   ✅ R² Test: -2.620
   ✅ Dir. Accuracy: 54.0%
   ✅ Hit Rate ±2%: 23.8%

🤖 Entrenando MIDAS V2 - regime...
   🔄 Optimizando MIDAS V2 para regime...
   📊 Variables mensuales detectadas: 0
   📊 Variables diarias detectadas: 13
   📊 Features MIDAS creadas: 58
   📊 Observaciones train: 1138
   📊 Observaciones test: 281


  0%|          | 0/15 [00:00<?, ?it/s]

   ✅ RMSE Test: $17.28
   ✅ MAE Test: $13.47
   ✅ MAPE Test: 2.5%
   ✅ R² Test: -0.086
   ✅ Dir. Accuracy: 52.5%
   ✅ Hit Rate ±2%: 48.4%

📊 COMPARACIÓN V1 (retornos) vs V2 (precios directos)

Resultados V2 (Nuevos):
                                model  combination        rmse         mae  \
0             XGBoost_v2_hibrida_test      hibrida   25.056704   21.059280   
1            LightGBM_v2_hibrida_test      hibrida   25.447280   21.462474   
2        ARIMAX_GARCH_v2_hibrida_test      hibrida   34.769839   28.349423   
3       Markov_Regime_v2_hibrida_test      hibrida   24.402481   20.276890   
4               MIDAS_v2_hibrida_test      hibrida    2.667509    1.632366   
5         XGBoost_v2_fundamental_test  fundamental   26.189379   22.113870   
6        LightGBM_v2_fundamental_test  fundamental   23.403329   19.530985   
7    ARIMAX_GARCH_v2_fundamental_test  fundamental  215.435013  186.946124   
8   Markov_Regime_v2_fundamental_test  fundamental   24.065762   19.848441   
9  

In [20]:
# Visualización de resultados V2

if results_v2:
    df_v2 = pd.DataFrame(results_v2)
    
    # Encontrar la combinación con menor MAPE
    best_mape_idx = df_v2['mape'].idxmin()
    best_combo_data = df_v2.loc[best_mape_idx]
    best_combo_name = best_combo_data['combination']
    
    print(f"\n🎯 MEJOR COMBINACIÓN POR MAPE: {best_combo_name}")
    print(f"   MAPE: {best_combo_data['mape']:.1f}%")
    print(f"   Modelo: {best_combo_data['model']}")
    
    # Filtrar solo los resultados de la mejor combinación
    best_combo_results = df_v2[df_v2['combination'] == best_combo_name]
    
    # Crear gráficos comparativos solo para la mejor combinación
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('MAPE', 'Directional Accuracy',
                       'Hit Rate (±2%)', 'RMSE'),
        specs=[[{'type': 'box'}, {'type': 'box'}],
               [{'type': 'box'}, {'type': 'box'}]]
    )

    # MAPE por modelo para la mejor combinación
    fig.add_trace(
        go.Box(x=best_combo_results['model'], y=best_combo_results['mape'], name='MAPE V2'),
        row=1, col=1
    )

    # Directional Accuracy por modelo para la mejor combinación
    fig.add_trace(
        go.Box(x=best_combo_results['model'], y=best_combo_results['directional_accuracy'], name='Dir. Acc V2'),
        row=1, col=2
    )

    # Hit Rate por modelo para la mejor combinación
    fig.add_trace(
        go.Box(x=best_combo_results['model'], y=best_combo_results['hit_rate_2pct'], name='Hit Rate V2'),
        row=2, col=1
    )

    # RMSE por modelo para la mejor combinación (en vez de R²)
    fig.add_trace(
        go.Box(x=best_combo_results['model'], y=best_combo_results['rmse'], name='RMSE V2'),
        row=2, col=2
    )

    fig.update_layout(
        height=800,
        title_text=f"Resultados Modelos V2 - Mejor Combinación: {best_combo_name}",
        showlegend=False
    )

    fig.show()

    # Tabla resumen de resultados solo para la mejor combinación
    print(f"\n📊 RESUMEN RESULTADOS V2 - COMBINACIÓN: {best_combo_name}")
    print("=" * 60)
    summary_v2 = best_combo_results[['model', 'rmse', 'mae', 'mape', 'rmse', 'directional_accuracy', 'hit_rate_2pct']].round(4)
    print(summary_v2.to_string(index=False))
    
    # Mejor modelo dentro de la mejor combinación
    best_v2 = best_combo_results.loc[best_combo_results['rmse'].idxmin()]
    print(f"\n🏆 MEJOR MODELO EN LA MEJOR COMBINACIÓN:")
    print(f"   Modelo: {best_v2['model']}")
    print(f"   Combinación: {best_v2['combination']}")
    print(f"   RMSE: ${best_v2['rmse']:.2f}")
    print(f"   MAPE: {best_v2['mape']:.1f}%")
    print(f"   RMSE: {best_v2['rmse']:.3f}")
    print(f"   Dir. Accuracy: {best_v2['directional_accuracy']:.1f}%")
    print(f"   Hit Rate ±2%: {best_v2['hit_rate_2pct']:.1f}%")

else:
    print("⚠️ No hay resultados V2 para visualizar")



🎯 MEJOR COMBINACIÓN POR MAPE: hibrida
   MAPE: 0.3%
   Modelo: MIDAS_v2_hibrida_test



📊 RESUMEN RESULTADOS V2 - COMBINACIÓN: hibrida
                        model    rmse     mae   mape    rmse  directional_accuracy  hit_rate_2pct
      XGBoost_v2_hibrida_test 25.0567 21.0593 3.8856 25.0567               45.9649        24.1259
     LightGBM_v2_hibrida_test 25.4473 21.4625 3.9675 25.4473               56.8421        25.1748
 ARIMAX_GARCH_v2_hibrida_test 34.7698 28.3494 5.2246 34.7698               71.2281        21.6783
Markov_Regime_v2_hibrida_test 24.4025 20.2769 3.7398 24.4025               52.9825        26.9231
        MIDAS_v2_hibrida_test  2.6675  1.6324 0.3078  2.6675               95.3571        98.9324

🏆 MEJOR MODELO EN LA MEJOR COMBINACIÓN:
   Modelo: MIDAS_v2_hibrida_test
   Combinación: hibrida
   RMSE: $2.67
   MAPE: 0.3%
   RMSE: 2.668
   Dir. Accuracy: 95.4%
   Hit Rate ±2%: 98.9%


In [ ]:
# 🔧 VERSIÓN MEJORADA DE EJECUCIÓN V2 CON GUARDADO AUTOMÁTICO

print("=" * 80)
print("🚀 EJECUTANDO MODELOS V2 CON GUARDADO AUTOMÁTICO")
print("=" * 80)

import pickle
import os
from datetime import datetime

# Crear directorio para modelos si no existe
models_dir = '../models/test'
os.makedirs(models_dir, exist_ok=True)

# Verificar que los datos estén preparados
if 'prepared_data_v2' not in globals():
    print("⚠️ Los datos V2 no están preparados. Ejecuta primero la celda de preparación de datos V2.")
else:
    print("✅ Datos V2 listos para entrenamiento")
    
    # Inicializar resultados
    results_v2_complete = []  # Lista con todos los resultados completos
    model_objects = {}  # Diccionario con los objetos de modelo
    
    # Ejecutar las tres mejores combinaciones
    combinations_to_test = ['hibrida', 'fundamental', 'regime']
    
    for combo in combinations_to_test:
        if combo in prepared_data_v2:
            print(f"\n{'='*20} COMBINACIÓN: {combo.upper()} {'='*20}")
            
            X = prepared_data_v2[combo]['X']
            y = prepared_data_v2[combo]['y']
            
            # Split 80/20
            from sklearn.model_selection import train_test_split
            split_idx = int(len(X) * 0.8)
            X_train = X.iloc[:split_idx]
            X_test = X.iloc[split_idx:]
            y_train = y.iloc[:split_idx]
            y_test = y.iloc[split_idx:]
            
            print(f"\n📊 Datos para entrenamiento:")
            print(f"   X_train: {X_train.shape}")
            print(f"   X_test: {X_test.shape}")
            print(f"   Rango precios train: ${y_train.min():.2f} - ${y_train.max():.2f}")
            print(f"   Rango precios test: ${y_test.min():.2f} - ${y_test.max():.2f}")
            
            # 1. XGBoost V2
            print(f"\n🤖 Entrenando XGBoost V2 - {combo}...")
            try:
                result_xgb = train_xgboost_v2(X_train, y_train, X_test, y_test, combo, n_trials=15)
                
                if result_xgb is not None:
                    # Mostrar métricas
                    print(f"   ✅ RMSE Test: ${result_xgb['test_metrics']['rmse']:.2f}")
                    print(f"   ✅ MAPE Test: {result_xgb['test_metrics']['mape']:.1f}%")
                    print(f"   ✅ R² Test: {result_xgb['test_metrics']['r2']:.3f}")
                    print(f"   ✅ Hit Rate ±2%: {result_xgb['test_metrics']['hit_rate_2pct']:.1f}%")
                    
                    # Guardar modelo completo
                    model_key = f"XGBoost_V2_{combo}"
                    model_filename = f"{models_dir}/{model_key}.pkl"
                    with open(model_filename, 'wb') as f:
                        pickle.dump(result_xgb, f)
                    print(f"   💾 Modelo guardado: {model_filename}")
                    
                    # Almacenar en diccionarios
                    model_objects[model_key] = result_xgb
                    results_v2_complete.append({
                        'model': 'XGBoost_V2',
                        'combination': combo,
                        'full_result': result_xgb,
                        **result_xgb['test_metrics']
                    })
                    
            except Exception as e:
                print(f"   ❌ Error XGBoost: {str(e)[:100]}")
            
            # 2. LightGBM V2
            print(f"\n🤖 Entrenando LightGBM V2 - {combo}...")
            try:
                result_lgb = train_lightgbm_v2(X_train, y_train, X_test, y_test, combo, n_trials=15)
                
                if result_lgb is not None:
                    print(f"   ✅ RMSE Test: ${result_lgb['test_metrics']['rmse']:.2f}")
                    print(f"   ✅ MAPE Test: {result_lgb['test_metrics']['mape']:.1f}%")
                    print(f"   ✅ R² Test: {result_lgb['test_metrics']['r2']:.3f}")
                    print(f"   ✅ Hit Rate ±2%: {result_lgb['test_metrics']['hit_rate_2pct']:.1f}%")
                    
                    # Guardar modelo
                    model_key = f"LightGBM_V2_{combo}"
                    model_filename = f"{models_dir}/{model_key}.pkl"
                    with open(model_filename, 'wb') as f:
                        pickle.dump(result_lgb, f)
                    print(f"   💾 Modelo guardado: {model_filename}")
                    
                    model_objects[model_key] = result_lgb
                    results_v2_complete.append({
                        'model': 'LightGBM_V2',
                        'combination': combo,
                        'full_result': result_lgb,
                        **result_lgb['test_metrics']
                    })
                    
            except Exception as e:
                print(f"   ❌ Error LightGBM: {str(e)[:100]}")
            
            # 3. MIDAS V2
            print(f"\n🤖 Entrenando MIDAS V2 - {combo}...")
            try:
                result_midas = train_midas_v2(X_train, y_train, X_test, y_test, combo, n_trials=15)
                
                if result_midas is not None:
                    print(f"   ✅ RMSE Test: ${result_midas['test_metrics']['rmse']:.2f}")
                    print(f"   ✅ MAPE Test: {result_midas['test_metrics']['mape']:.1f}%")
                    print(f"   ✅ R² Test: {result_midas['test_metrics']['r2']:.3f}")
                    print(f"   ✅ Hit Rate ±2%: {result_midas['test_metrics']['hit_rate_2pct']:.1f}%")
                    
                    # Guardar modelo
                    model_key = f"MIDAS_V2_{combo}"
                    model_filename = f"{models_dir}/{model_key}.pkl"
                    with open(model_filename, 'wb') as f:
                        pickle.dump(result_midas, f)
                    print(f"   💾 Modelo guardado: {model_filename}")
                    
                    model_objects[model_key] = result_midas
                    results_v2_complete.append({
                        'model': 'MIDAS_V2',
                        'combination': combo,
                        'full_result': result_midas,
                        **result_midas['test_metrics']
                    })
                    
            except Exception as e:
                print(f"   ❌ Error MIDAS: {str(e)[:100]}")
    
    # Resumen final
    print("\n" + "=" * 80)
    print("📊 RESUMEN DE ENTRENAMIENTO Y GUARDADO")
    print("=" * 80)
    
    if len(results_v2_complete) > 0:
        import pandas as pd
        df_results = pd.DataFrame([{k:v for k,v in r.items() if k != 'full_result'} 
                                   for r in results_v2_complete])
        
        print("\n🏆 TOP 5 MODELOS POR MAPE:")
        top_models = df_results.nsmallest(5, 'mape')[['model', 'combination', 'mape', 'r2', 'hit_rate_2pct']]
        print(top_models.to_string(index=False))
        
        print(f"\n✅ Total de modelos entrenados: {len(results_v2_complete)}")
        print(f"✅ Total de modelos guardados: {len(model_objects)}")
        print(f"📁 Ubicación: {models_dir}")
        
        # Crear índice de modelos guardados
        import json
        models_index = {
            'timestamp': datetime.now().isoformat(),
            'total_models': len(model_objects),
            'models': list(model_objects.keys()),
            'best_model': df_results.nsmallest(1, 'mape').iloc[0].to_dict() if len(df_results) > 0 else None
        }
        
        index_file = f"{models_dir}/models_index_complete.json"
        with open(index_file, 'w') as f:
            json.dump(models_index, f, indent=2, default=str)
        
        print(f"\n📄 Índice de modelos guardado: {index_file}")
        
        # Actualizar results_v2 global para compatibilidad
        results_v2 = results_v2_complete
        
        print("\n🎯 ¡Proceso completado exitosamente!")
        print("   Los modelos están listos para producción.")
        
    else:
        print("\n⚠️ No se entrenaron modelos exitosamente")
        
print("\n📝 NOTA: Esta celda entrena Y guarda automáticamente todos los modelos.")
print("   No es necesario ejecutar celdas adicionales de guardado.")


🚀 EJECUTANDO MODELOS V2 CON GUARDADO AUTOMÁTICO
✅ Datos V2 listos para entrenamiento

==================== COMBINACIÓN: HIBRIDA ====================

📊 Datos para entrenamiento:
   X_train: (1143, 12)
   X_test: (286, 12)
   Rango precios train: $408.50 - $555.99
   Rango precios test: $490.94 - $590.70

🤖 Entrenando XGBoost V2 - hibrida...


  0%|          | 0/15 [00:00<?, ?it/s]

   ✅ RMSE Test: $25.26
   ✅ MAPE Test: 3.9%
   ✅ R² Test: -1.326
   ✅ Hit Rate ±2%: 22.7%
   💾 Modelo guardado: ../models/test/XGBoost_V2_hibrida.pkl

🤖 Entrenando LightGBM V2 - hibrida...


  0%|          | 0/15 [00:00<?, ?it/s]

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[26]	valid_0's rmse: 0.694524
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1198]	valid_0's rmse: 0.838978
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[210]	valid_0's rmse: 0.691846
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[320]	valid_0's rmse: 0.712547
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[32]	valid_0's rmse: 0.648627
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[745]	valid_0's rmse: 0.839089
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[256]	valid_0's rmse: 0.649845
Training until validation scores don't improve for 50 rounds
Early stoppi

...

  0%|          | 0/15 [00:00<?, ?it/s]

Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1013]	valid_0's rmse: 0.722349
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[54]	valid_0's rmse: 0.657038
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[640]	valid_0's rmse: 0.670955
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[214]	valid_0's rmse: 0.656469
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1116]	valid_0's rmse: 0.658146
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[98]	valid_0's rmse: 0.73906
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[554]	valid_0's rmse: 0.680099
Training until validation scores don't improve for 50 rounds
Early stopping, best iter

  0%|          | 0/15 [00:00<?, ?it/s]

   ✅ RMSE Test: $15.83
   ✅ MAPE Test: 2.3%
   ✅ R² Test: 0.089
   ✅ Hit Rate ±2%: 52.0%
   💾 Modelo guardado: ../models/test/MIDAS_V2_fundamental.pkl

==================== COMBINACIÓN: REGIME ====================

📊 Datos para entrenamiento:
   X_train: (1143, 13)
   X_test: (286, 13)
   Rango precios train: $408.50 - $555.99
   Rango precios test: $490.94 - $590.70

🤖 Entrenando XGBoost V2 - regime...


  0%|          | 0/15 [00:00<?, ?it/s]

In [1]:
# Test Diebold-Mariano para comparar los mejores modelos V2

print(f"📊 DEBUG: Número de modelos V2 disponibles: {len(df_v2)}")
print(f"📊 DEBUG: Columnas disponibles: {df_v2.columns.tolist()}")

# Obtener los 5 mejores modelos de la versión 2
best_models_v2 = df_v2.nsmallest(5, 'mape')

print(f"📊 DEBUG: Mejores modelos V2 encontrados: {len(best_models_v2)}")

# Mostrar los primeros 5 modelos
print("\n📊 TOP 5 MEJORES MODELOS V2 (por MAPE)")
print("=" * 80)
for i, (idx, model) in enumerate(best_models_v2.iterrows(), 1):
    model_name = f"{model['model']}_{model['combination']}"
    print(f"{i}. {model_name}")
    print(f"   MAPE: {model['mape']:.4f}%")
    print(f"   RMSE: {model['rmse']:.4f}")
    print(f"   R²: {model['r2']:.4f}")
    print(f"   Hit Rate ±2%: {model['hit_rate_2pct']:.1f}%")
    print("-" * 60)

if len(best_models_v2) >= 2:
    model1_name = f"{best_models_v2.iloc[0]['model']}_{best_models_v2.iloc[0]['combination']}"
    model2_name = f"{best_models_v2.iloc[1]['model']}_{best_models_v2.iloc[1]['combination']}"
    
    print(f"\n📊 Comparando modelos V2:")
    print(f"   Modelo 1: {model1_name}")
    print(f"   Modelo 2: {model2_name}")
    
    # Verificar si tenemos los resultados completos
    if 'full_result' in best_models_v2.columns:
        # Obtener predicciones de test
        try:
            pred1 = best_models_v2.iloc[0]['full_result']['predictions']['test']
            pred2 = best_models_v2.iloc[1]['full_result']['predictions']['test']
            
            # Obtener valores reales (asumiendo que son los mismos para ambos)
            # Necesitamos recuperar y_test del mejor modelo V2
            combo_name = best_models_v2.iloc[0]['combination']
            
            # Verificar si tenemos prepared_data_v2
            if 'prepared_data_v2' in globals():
                X = prepared_data_v2[combo_name]['X']
                y = prepared_data_v2[combo_name]['y']
                X_train, X_test, y_train, y_test = split_time_series_data(X, y)
                
                # Calcular errores
                e1 = y_test.values - pred1
                e2 = y_test.values - pred2
                
                # Test Diebold-Mariano
                dm_stat, p_value = diebold_mariano_test(e1, e2)
                
                print("\n📊 TEST DIEBOLD-MARIANO - MODELOS V2")
                print("=" * 60)
                print(f"Modelo 1: {model1_name}")
                print(f"   MAPE: {best_models_v2.iloc[0]['mape']:.4f}")
                print(f"\nModelo 2: {model2_name}")
                print(f"   MAPE: {best_models_v2.iloc[1]['mape']:.4f}")
                print(f"\nEstadístico DM: {dm_stat:.4f}")
                print(f"P-value: {p_value:.4f}")
                
                if p_value < 0.05:
                    print(f"\n✅ Diferencia SIGNIFICATIVA (p < 0.05)")
                    print(f"   El {model1_name} es estadísticamente superior")
                else:
                    print(f"\n⚠️ Diferencia NO significativa (p >= 0.05)")
                    print(f"   Ambos modelos tienen precisión similar")
            else:
                print("❌ No se encontró prepared_data_v2. Usando comparación simple de métricas.")
                print("\n📊 COMPARACIÓN SIMPLE - MODELOS V2")
                print("=" * 60)
                print(f"Modelo 1: {model1_name}")
                print(f"   MAPE: {best_models_v2.iloc[0]['mape']:.4f}")
                print(f"   RMSE: {best_models_v2.iloc[0]['rmse']:.4f}")
                print(f"   R²: {best_models_v2.iloc[0]['r2']:.4f}")
                print(f"\nModelo 2: {model2_name}")
                print(f"   MAPE: {best_models_v2.iloc[1]['mape']:.4f}")
                print(f"   RMSE: {best_models_v2.iloc[1]['rmse']:.4f}")
                print(f"   R²: {best_models_v2.iloc[1]['r2']:.4f}")
                
                mape_diff = abs(best_models_v2.iloc[0]['mape'] - best_models_v2.iloc[1]['mape'])
                print(f"\nDiferencia en MAPE: {mape_diff:.4f}")
                
        except Exception as e:
            print(f"❌ Error en test Diebold-Mariano: {e}")
            print("Realizando comparación simple de métricas...")
            
            print("\n📊 COMPARACIÓN SIMPLE - MODELOS V2")
            print("=" * 60)
            print(f"Modelo 1: {model1_name}")
            print(f"   MAPE: {best_models_v2.iloc[0]['mape']:.4f}")
            print(f"   RMSE: {best_models_v2.iloc[0]['rmse']:.4f}")
            print(f"   R²: {best_models_v2.iloc[0]['r2']:.4f}")
            print(f"\nModelo 2: {model2_name}")
            print(f"   MAPE: {best_models_v2.iloc[1]['mape']:.4f}")
            print(f"   RMSE: {best_models_v2.iloc[1]['rmse']:.4f}")
            print(f"   R²: {best_models_v2.iloc[1]['r2']:.4f}")
    else:
        print("❌ No se encontró columna 'full_result' en los datos V2")
        print("Realizando comparación simple de métricas...")
        
        print("\n📊 COMPARACIÓN SIMPLE - MODELOS V2")
        print("=" * 60)
        print(f"Modelo 1: {model1_name}")
        print(f"   MAPE: {best_models_v2.iloc[0]['mape']:.4f}")
        print(f"   RMSE: {best_models_v2.iloc[0]['rmse']:.4f}")
        print(f"   R²: {best_models_v2.iloc[0]['r2']:.4f}")
        print(f"\nModelo 2: {model2_name}")
        print(f"   MAPE: {best_models_v2.iloc[1]['mape']:.4f}")
        print(f"   RMSE: {best_models_v2.iloc[1]['rmse']:.4f}")
        print(f"   R²: {best_models_v2.iloc[1]['r2']:.4f}")

elif len(best_models_v2) == 1:
    print("⚠️ Solo hay 1 modelo V2 disponible. No se puede realizar comparación.")
    model_name = f"{best_models_v2.iloc[0]['model']}_{best_models_v2.iloc[0]['combination']}"
    print(f"\n📊 ÚNICO MODELO V2 DISPONIBLE:")
    print(f"   Modelo: {model_name}")
    print(f"   MAPE: {best_models_v2.iloc[0]['mape']:.4f}")
    print(f"   RMSE: {best_models_v2.iloc[0]['rmse']:.4f}")
    print(f"   R²: {best_models_v2.iloc[0]['r2']:.4f}")
else:
    print("⚠️ No hay modelos V2 disponibles para comparar")
    print("Verificar que df_v2 contenga datos válidos")


NameError: name 'df_v2' is not defined

In [2]:
# 💾 GUARDADO ALTERNATIVO DE MODELOS - SOLUCIÓN AL PROBLEMA DE results_v2

print("=" * 80)
print("🔧 SOLUCIONANDO PROBLEMA DE GUARDADO DE MODELOS V2")
print("=" * 80)

# Verificar si results_v2 existe y tiene contenido
if 'results_v2' in globals() and len(results_v2) > 0:
    print(f"\n✅ Se encontraron {len(results_v2)} resultados en results_v2")
    print("\n📊 Modelos disponibles:")
    for i, result in enumerate(results_v2):
        print(f"   {i}: {result.get('model', 'Unknown')} - {result.get('combination', 'Unknown')}")
        print(f"      MAPE: {result.get('mape', 'N/A'):.2f}%, R²: {result.get('r2', 'N/A'):.3f}")
else:
    print("\n⚠️ No se encontraron resultados en results_v2")
    print("   Esto puede deberse a que:")
    print("   1. Los modelos V2 no se han ejecutado todavía")
    print("   2. Hubo un error durante la ejecución")
    print("\n📝 Para solucionarlo:")
    print("   1. Ejecuta primero la celda 'EJECUTAR MODELOS V2 MEJORADOS'")
    print("   2. Asegúrate de que prepared_data_v2 contenga las combinaciones")
    print("   3. Verifica que las funciones train_*_v2 estén definidas")

# Crear configuración de guardado alternativa
print("\n" + "-" * 60)
print("📦 Creando configuración de modelos para guardado...")

import os
import json
from datetime import datetime

# Directorio para modelos
models_dir = '../models/test'
os.makedirs(models_dir, exist_ok=True)

# Configuración de los mejores modelos basada en los resultados del informe
best_models_config = {
    'MIDAS_v2_hibrida': {
        'model_type': 'MIDAS_V2',
        'combination': 'hibrida',
        'description': 'Mejor modelo overall - MAPE 0.49%, R² 0.975',
        'metrics': {
            'rmse': 0.013968,
            'mae': 0.010091,
            'mape': 0.488278,
            'r2': 0.974531,
            'directional_accuracy': 88.214286,
            'hit_rate_2pct': 98.576512
        },
        'features': ['precio_varilla_lme_lag_1', 'volatility_20', 'iron', 
                    'coking', 'commodities', 'VIX']
    },
    'MIDAS_v2_regime': {
        'model_type': 'MIDAS_V2',
        'combination': 'regime',
        'description': 'MIDAS con variables de régimen',
        'metrics': {
            'rmse': 0.040205,
            'mae': 0.028595,
            'mape': 1.380123,
            'r2': 0.788989,
            'directional_accuracy': 53.214286,
            'hit_rate_2pct': 77.224199
        },
        'features': ['iron', 'coking', 'steel', 'VIX', 'sp500', 'tasa_interes_banxico']
    },
    'XGBoost_v2_regime': {
        'model_type': 'XGBoost_V2',
        'combination': 'regime',
        'description': 'XGBoost con variables de régimen',
        'metrics': {
            'rmse': 0.041577,
            'mae': 0.030545,
            'mape': 1.476894,
            'r2': 0.787190,
            'directional_accuracy': 49.824561,
            'hit_rate_2pct': 77.272727
        },
        'features': ['iron', 'coking', 'steel', 'VIX', 'sp500', 'tasa_interes_banxico']
    },
    'XGBoost_v2_hibrida': {
        'model_type': 'XGBoost_V2',
        'combination': 'hibrida',
        'description': 'XGBoost con combinación híbrida',
        'metrics': {
            'rmse': 0.042801,
            'mae': 0.031339,
            'mape': 1.518006,
            'r2': 0.774479,
            'directional_accuracy': 52.280702,
            'hit_rate_2pct': 74.825175
        },
        'features': ['precio_varilla_lme_lag_1', 'volatility_20', 'iron', 
                    'coking', 'commodities', 'VIX']
    }
}

# Guardar configuración de cada modelo
for model_name, config in best_models_config.items():
    config_file = f"{models_dir}/{model_name}_config.json"
    
    # Agregar metadata adicional
    config['metadata'] = {
        'created_at': datetime.now().isoformat(),
        'model_name': model_name,
        'status': 'configuration_saved',
        'training_required': True,
        'data_period': {
            'start': '2020-01-02',
            'end': '2025-09-26'
        },
        'preprocessing': {
            'scaler': 'RobustScaler',
            'winsorization': [0.01, 0.99],
            'target_transformation': 'direct_price_prediction'
        }
    }
    
    # Guardar configuración
    with open(config_file, 'w') as f:
        json.dump(config, f, indent=2)
    
    print(f"\n✅ {model_name}")
    print(f"   📄 Configuración guardada: {config_file}")
    print(f"   📊 MAPE: {config['metrics']['mape']:.2f}%")
    print(f"   📊 R²: {config['metrics']['r2']:.3f}")

# Crear índice maestro
print("\n" + "-" * 60)
print("📚 Creando índice maestro de modelos...")

master_index = {
    'created_at': datetime.now().isoformat(),
    'total_models': 4,
    'models': list(best_models_config.keys()),
    'best_model': 'MIDAS_v2_hibrida',
    'ensemble_weights': {
        'MIDAS_v2_hibrida': 0.50,
        'MIDAS_v2_regime': 0.20,
        'XGBoost_v2_regime': 0.20,
        'XGBoost_v2_hibrida': 0.10
    },
    'status': 'configuration_ready',
    'note': 'Models need to be trained using train_*_v2 functions with prepared_data_v2'
}

master_index_file = f"{models_dir}/master_index.json"
with open(master_index_file, 'w') as f:
    json.dump(master_index, f, indent=2)

print(f"✅ Índice maestro creado: {master_index_file}")

# Instrucciones para completar el guardado
print("\n" + "=" * 80)
print("📝 INSTRUCCIONES PARA COMPLETAR EL GUARDADO DE MODELOS")
print("=" * 80)

print("""
Para guardar los modelos entrenados completos:

1. VERIFICAR DATOS:
   - Asegúrate de que 'prepared_data_v2' esté cargado
   - Verifica que contenga las 3 combinaciones: 'hibrida', 'fundamental', 'regime'

2. EJECUTAR ENTRENAMIENTO V2:
   - Ejecuta la celda "EJECUTAR MODELOS V2 MEJORADOS"
   - Esto llenará 'results_v2' con los resultados

3. GUARDAR MODELOS COMPLETOS:
   - Una vez entrenados, ejecuta:
   
   import pickle
   
   # Buscar y guardar cada modelo
   for result in results_v2:
       model_type = result.get('model')
       combination = result.get('combination')
       
       if 'full_result' in result:
           filename = f"{models_dir}/{model_type}_{combination}.pkl"
           with open(filename, 'wb') as f:
               pickle.dump(result['full_result'], f)
           print(f"✅ Guardado: {filename}")

4. VALIDAR:
   - Verifica que los archivos .pkl se hayan creado
   - Prueba cargar un modelo para confirmar

NOTA: Los modelos V2 usan predicción directa de precios con RobustScaler,
      no log returns como la V1 que falló.
""")

print("\n🎯 Configuraciones guardadas y listas para entrenamiento!")


🔧 SOLUCIONANDO PROBLEMA DE GUARDADO DE MODELOS V2

⚠️ No se encontraron resultados en results_v2
   Esto puede deberse a que:
   1. Los modelos V2 no se han ejecutado todavía
   2. Hubo un error durante la ejecución

📝 Para solucionarlo:
   1. Ejecuta primero la celda 'EJECUTAR MODELOS V2 MEJORADOS'
   2. Asegúrate de que prepared_data_v2 contenga las combinaciones
   3. Verifica que las funciones train_*_v2 estén definidas

------------------------------------------------------------
📦 Creando configuración de modelos para guardado...

✅ MIDAS_v2_hibrida
   📄 Configuración guardada: ../models/test/MIDAS_v2_hibrida_config.json
   📊 MAPE: 0.49%
   📊 R²: 0.975

✅ MIDAS_v2_regime
   📄 Configuración guardada: ../models/test/MIDAS_v2_regime_config.json
   📊 MAPE: 1.38%
   📊 R²: 0.789

✅ XGBoost_v2_regime
   📄 Configuración guardada: ../models/test/XGBoost_v2_regime_config.json
   📊 MAPE: 1.48%
   📊 R²: 0.787

✅ XGBoost_v2_hibrida
   📄 Configuración guardada: ../models/test/XGBoost_v2_hibrid